In [ ]:
import pygame
import sys
# 7:00:27 / 7:38:56 'da kaldım. Player içerisinde get_cost_by_index ve get_values_by_index metotlarını oluşturup her bir rect'i çizerken alacağım değerleri index ile seçeceğim.

In [ ]:
# Settings

WIDTH = 1280 # Ekran genişliği
HEIGHT = 720 # Ekran boyutu
FPS = 60 # FPS
TILESIZE = 64 # Her bir parçanın genişliği kaç px olsun.

# Nesneler için farklı hitbox değerleri
HITBOX_OFFSET = {
    'player': -26,
    'object': -40,
    'grass': -10,
    'invisible': 0
}

# ui bilgileri
BAR_HEIGHT = 20 # UI bar uzunluğu
HEALTH_BAR_WIDTH = 200 # Can barı genişliği
ENERGY_BAR_WIDTH = 140 # Enerji barı genişiliği
ITEM_BOX_SIZE = 80 # Silah ve büyü çerçevesi
UI_FONT = 'graphics/font/joystix.ttf' # UI fontu
UI_FONT_SIZE = 18 # UI font yazı büyüklüğü

# genel renkler
WATER_COLOR = '#71ddee' # Su rengi
UI_BG_COLOR = '#222222' # UI arkaplan rengi
UI_BORDER_COLOR = '#111111' # UI çerçeve rengi
TEXT_COLOR = '#EEEEEE' # Text rengi

# ui renkleri
HEALTH_COLOR = 'red' # Can barı rengi
ENERGY_COLOR = 'blue' # Enerji barı rengi
UI_BORDER_COLOR_ACTIVE = 'gold' # UI aktif çerçeve rengi

# geliştirme ekranı bilgileri
TEXT_COLOR_SELECTED = "#111111" # Seçili text rengi
BAR_COLOR = "#EEEEEE" # Bar rengi
BAR_COLOR_SELECTED = "#111111" # Seçili bar rengi
UPGRADE_BG_COLOR_SELECTED = "#EEEEEE" # Seçili arkaplan rengi

# silah bilgileri
weapon_data = { # Silahlarımızı içeren bir data.
    'sword': {'cooldown':100, 'damage':15, 'graphic': 'graphics/weapons/sword/full.png'},
    'lance': {'cooldown':400, 'damage':30, 'graphic': 'graphics/weapons/lance/full.png'},
    'axe': {'cooldown':300, 'damage':20, 'graphic': 'graphics/weapons/axe/full.png'},
    'rapier': {'cooldown':50, 'damage':8, 'graphic': 'graphics/weapons/rapier/full.png'},
    'sai': {'cooldown':80, 'damage':10, 'graphic': 'graphics/weapons/sai/full.png'},
}

# büyü bilgileri
magic_data = { # Büyü bilgilerini tutan bir data.
    'flame': {'strength':5,'cost':20,'graphic':'graphics/particles/flame/fire.png'},
    'heal': {'strength':20,'cost':10,'graphic':'graphics/particles/heal/heal.png'}
}

# düşmanlar
monster_data = { # Düşman bilgilerini tutan data.
    'squid': {'health':100, 'exp':100, 'damage':20, 'attack_type':'slash', 'attack_sound':'audio/attack/slash.wav', 'speed':3, 'resistance':3, 'attack_radius':80, 'notice_radius':360},
    'raccoon': {'health':300, 'exp':250, 'damage':40, 'attack_type':'claw', 'attack_sound':'audio/attack/claw.wav', 'speed':2, 'resistance':3, 'attack_radius':120, 'notice_radius':400},
    'spirit': {'health':100, 'exp':110, 'damage':8, 'attack_type':'thunder', 'attack_sound':'audio/attack/fireball.wav', 'speed':4, 'resistance':3, 'attack_radius':60, 'notice_radius':350},
    'bamboo': {'health':70, 'exp':120, 'damage':6, 'attack_type':'leaf_attack', 'attack_sound':'audio/attack/slash.wav', 'speed':3, 'resistance':3, 'attack_radius':50, 'notice_radius':300},
} # resistance düşmana vurulduğunda arkaya fırlatılma değeridir.

# WORLD_MAP = [ # Oyun dünyamız Her bir karakter arası TILESIZE = 64 px olacak. Artık gerek yok.
# ['x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x'],
# ['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
# ['x',' ','p',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ','x',' ',' ',' ',' ',' ','x','x','x','x','x',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x','x','x',' ',' ',' ','x'],
# ['x',' ',' ',' ',' ',' ',' ','x',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ',' ',' ',' ','x','x','x','x','x',' ',' ',' ',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ',' ',' ',' ',' ','x','x','x',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ',' ',' ',' ',' ',' ','x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
# ['x',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ',' ','x'],
# ['x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x','x'],
# ]

In [ ]:
# Main

import pygame, sys
from settings import *
from level import Level

class Game:
  def __init__(self):

    # Genel ayarlar.
    pygame.init() # Pygame modülünü başlatıyoruz.
    self.screen = pygame.display.set_mode((WIDTH,HEIGHT)) # Bir ekran oluşturuyoruz.
    pygame.display.set_caption("ZeldaLike") # Pencere ismini veriyoruz.
    self.clock = pygame.time.Clock() # bir Clock nesnesi oluşturuyoruz.

    self.level = Level() # Bir level nesnesi oluşturuyoruz.

    # Ses
    main_sound = pygame.mixer.Sound('audio/main.ogg') # Arkaplan şarkısını oluşturduk.
    main_sound.set_volume(0.5) # Ve sesi ayarlıyoruz.
    main_sound.play(loops = -1) # Ve sonsuz çalmasını sağlıyoruz.

  def run(self): # Oyun sürerken gerçekleşecekleri yazacağımız metot.
    while True: # Sonsuz döngü
      for event in pygame.event.get(): # Tüm olayları getiriyoruz.
        if event.type == pygame.QUIT: # Eğer olayımız çıkış yapmaksa
          pygame.quit() # pygame'den çıkış yapıyoruz.
          sys.exit() # Ve tüm dosyalardan çıkış yapıyoruz.
        if event.type == pygame.KEYDOWN: # Eğer herhangi bir tuşa basıyorsak.
          if event.key == pygame.K_m: # Ve klavyede m tuşuna basıyorsak.
            self.level.toggle_menu() # Level içerisindeki toggle_menu metodunu çalıştır.

      self.screen.fill(WATER_COLOR) # Ekranı deniz rengine boyuyoruz ki çok kenara gidersek siyahlık görmeyelim.
      self.level.run() # level objesinden run metotunu çalıştırıyoruz. (Oyunu çizecek)
      pygame.display.update() # Update metodu ile ekranı sürekli güncelliyoruz.
      self.clock.tick(FPS) # Fps i veriyoruz. (Genelde 60 idealdir.)

if __name__ == "__main__": # Eğer main dosyasında isek
  game = Game() # bir oyun objesi oluştur
  game.run() # ve run metodunu çalıştır.

In [ ]:
# Level

# Level tüm spriteları içerecek.

# 2 farklı sprite grubu oluşturacağız.

# visible_sprites grubu oyuna çizilecek sprite ların bulunduğu grup olacak.
# obstacle_sprites grubu oyuna çizilmeyecek fakat oyuncu çarpışma yapabilecek. Oyun sınırları gibi

import pygame 
from settings import *
from tile import Tile
from player import Player
from debug import debug
from support import *
from random import choice, randint
from weapon import Weapon
from ui import UI
from enemy import Enemy
from particles import AnimationPlayer
from magic import MagicPlayer
from upgrade import Upgrade

class Level:
  def __init__(self):

    # Display ekranı oluşturacağız.
    self.display_surface = pygame.display.get_surface() # Yaptıklarımızı bir ekranda gösterecek nesne oluşturduk.
    self.game_paused = False # Oyunun durma durumu False ise yani oyun akıyor ise bir değişken oluşturduk.

    # Sprite grupları
     # self.visible_sprites = pygame.sprite.Group() # Oyunda görünecek varlıkların olduğu grup

    # visible_sprites'ı kendi oluşturduğumuz bir gruptan örnek yapmak istiyorum.

    self.visible_sprites = YSortCameraGroup() # Orjinal Group classının bir örneği olduğu için hala bir grup. Kendi oluşturduğumuz grubu veriyoruz.
    self.obstacle_sprites = pygame.sprite.Group() # Oyunda görünmeyen fakat çarpışma olacak varlıkların olduğu grup

    # saldırı sprite'ları
    self.current_attack = None # Ekranda silah nesnesinin bulunup bulunmadığını kontrol etmek için bir değişken oluşturuyoruz.
    self.attack_sprites = pygame.sprite.Group() # Silah ve büyülerin bulunacağı sprite grubu.
    self.attackable_sprites = pygame.sprite.Group() # Obje ve düşmanların bulunacağı sprite grubu.

    # sprite ayarlamaları
    self.create_map() # Dünyayı çizen metotu direkt Level objesinin init kısmında çağırıyoruz.

    # Kullanıcı arayüzü (UI)
    self.ui = UI()
    self.upgrade = Upgrade(self.player) # Upgrade sınıfından bir nesne oluşturduk.

    # Partiküller
    self.animation_player = AnimationPlayer() # Oyun başlangıcında implemente ediyoruz ki oyun yavaşlamasın.
    self.magic_player = MagicPlayer(self.animation_player) # MagicPlayer class'ından bir nesne oluşturduk ki burada da ulaşabilelim.

  def create_map(self): # Oyun dünyasını oluşturup çizecek metodu yazıyoruz.
  # 1- Eski deneme WORLD_MAP ' i çizdiren kısımları comment ediyoruz.
  """
    for row_index,row in enumerate(WORLD_MAP): # Dünya haritasındaki her bir liste ve o listenin indexi için
      # Row indexi bizim y konumu için 64 ile çarpacak olduğumuz index numarası yani ilk satır için 0 2. için 1 getirecek ve biz 64 ile çarpacağız.
      for col_index,col in enumerate(row): # Her bir satır içerisinde ise her bir kolona bakacağız böylece x değerleri için çarpacağımız index numarasını col_index şeklinde alacağız.
        x = col_index * TILESIZE # (64) # Her bir elemanın x pozisyonu
        y = row_index * TILESIZE # (64) # Her bir elemanın y pozisyonu
        if col == 'x': # Dünya haritamızdaki her bir x değeri bizim için bir taş çizeceğimiz anlamına geliyor.
          Tile((x,y),[self.visible_sprites,self.obstacle_sprites]) # Bir Tile oluşturuyoruz.(taş)
        if col == 'p': # Dünya haritamızdaki her bir p değeri bizim için bir oyuncu çizeceğimiz anlamına geliyor.
          self.player = Player((x,y),[self.visible_sprites],self.obstacle_sprites) # Bir Player oluşturuyoruz.(oyuncu)
          # Player'ın konumunu, bulunduğu grubu ve player'ın çarpışma yapabilmesi için gerekli objelerin bulunduğu sprite grubunu tanımlıyoruz.
  """
  # 2- CSV dosyasından bir map yükleyeceğiz. Yine aynı mantıkta dosyayı okuyup çizdireceğiz.
    layouts = { # Bir harita düzeni oluşturuyoruz.
        'boundary': import_csv_layout('map/map_FloorBlocks.csv'), # Sınırlarımız için csv dosyasını çalışılabilir hale getirecek bir metot yazacağız.
        # Metoda bir path vereceğiz ve bu dosyayı alıp işleyecek. İşledikten sonra aynı eski harita gibi yerleştirmek istiyorum.
        'grass': import_csv_layout('map/map_Grass.csv'), # csv dosyasını işleyip grassları oluşturacağız. Grassların konumunu alıyoruz aslında.
        'object': import_csv_layout('map/map_Objects.csv'), # csv dosyasını işleyip nesneleri oluşturacağız. Objectlerin konumunu alıuyoruz aslında.
        'entities': import_csv_layout('map/map_Entities.csv') # csv dosyasını işleyip entityleri oluşturacağız. Entitylerin konumunu alıyoruz aslında.
    }

    graphics = { # Bir grafik dictionary'si oluşturuyoruz.
        'grass': import_folder('graphics/Grass'), # Oluşturduğumuz metot ile verilen konumdaki tüm resimleri bir surface olarak getiriyoruz liste içerisinde.
        'objects': import_folder('graphics/objects') # Oluşturduğumuz metot ile verilen konumdaki tüm resimleri bir surface olarak getiriyoruz liste içerisinde.
    }

    for style,layout in layouts.items(): # Layout içerisindeli key'leri (boundary ve graphic gibi) style'a haritayı veren listeyi de layout'a atıyoruz.
      for row_index,row in enumerate(layout): # Layout içerisindeki her bir satır ve numarasını y ekseni için alıyoruz.
        for col_index,col in enumerate(row): # Her bir satırın kolon numarasını da x ekseni için alıyoruz.
          if col != '-1': # Eğer colon -1 değilse yani boş değil bir nesne var ise.
            x = col_index * TILESIZE # Her bir elemanın x pozisyonu
            y = row_index * TILESIZE # Her bir elemanın y pozisyonu

            if style == 'boundary': # Eğer style'ımız boundary ise yani sınır ise
              Tile((x,y),[self.obstacle_sprites],'invisible') # Yeni bir Tile oluşturuyoruz. (pos,group,sprite_type,surface parametrelerini alacak.)
              # x,y koordinatlarına ve görünmeyen obstacles_sprites grubuna atıyoruz. surface vermediğimiz için siyah bir blok oluşturacak fakat onu göremeyeceğiz.
            if style == 'grass': # Eğer grass ise style'ımız bir grass tile'ı oluşturacağız.
              random_grass_image = choice(graphics['grass']) # Random modülünden choice metodu yardımıyla rastgele bir surface seçiyoruz.
              Tile((x,y),[self.visible_sprites,self.obstacle_sprites,self.attackable_sprites],'grass',random_grass_image) # Ve grass tile'ımızı oluşturuyoruz.
              # Grass objesi aynı zamanda self.attackable_sprites içerisinde olmalı ki collision olduğunda yok olsun.
            if style == 'object': # Eğer object ise style'ımız bir object tile'ı oluşturacağız.
              surf = graphics['objects'][int(col)] # Tiles üzerinde her bir ağacın belli bir konumu var. Bu nedenle rastgele seçim yapamıyoruz.
              # Bunu önlemek için str kolon numarasını integer'a çevirerek o indeksteki fotoğrafı getiriyoruz. Yani liste içerisinden onu seçiyoruz.
              Tile((x,y),[self.visible_sprites,self.obstacle_sprites],'object',surf) # Ve objemizi oluşturuyoruz.
              # Fakat bu işlem nesneler büyük olduğu fakat Tile classında 64*64 kabul ettiğimiz için üst üste oluşumlar yaratıyor. Bu problemi o class içerisinde çözüyoruz.
            if style == 'entities': # Eğer style'imiz entities ise:
              if col == '394': # Ve colon numarası 394 ise (394=player)
                self.player = Player((x,y),[self.visible_sprites],self.obstacle_sprites,self.create_attack,self.destroy_attack,self.create_magic) # Metodu çalıştırmıyoruz sadece çağırıyoruz. İle player oluşturuyoruz.
              else: # Eğer değilse her biri için bir düşman oluştur.
                if col == '390': # Eğer layouttaki sayı 390 ise bir bamboo
                  monster_name = 'bamboo'
                elif col == '391': # Eğer 391 ise bir spirit
                  monster_name = 'spirit'
                elif col == '392': # Eğer 392 ise bir raccoon
                  monster_name = 'raccoon'
                else: # Eğer hiçbiri değilse bir squid oluşturmak için monster_name'leri ayarla.
                  monster_name = 'squid'
                Enemy(monster_name,(x,y),[self.visible_sprites,self.attackable_sprites],self.obstacle_sprites,self.damage_player,self.trigger_death_particles,self.add_exp) # Bir düşman nesnesi oluşturduk. Daha sonra her bir sayı değeri için farklı bir canavar oluşturacağız.
                # Ve tekrar collision için tüm obstacle_sprites'lara bakıyoruz ve bunu düşmanlar için de tanımlamalıyız.
                # sprite grupları arasına self.attackable_sprites'ı da ekliyoruz.
                # player'a hasar verebilmesini sağlayacak olan metodu da veriyoruz.
                # Ölüm animasyonu için bir metot daha veriyoruz.
                # add_exp ile öldüklerinde player'a exp vermelerini istiyoruz. Enemy içerisinde metodu çağıracağız.
    # 1-
     # self.player = Player((2000,1430),[self.visible_sprites],self.obstacle_sprites) # Karakterimizi çizdirdik ekrana.
    # 2- self.player oluştururken aşağıdaki create_attack metodunu da bir parametre olarak vermeliyiz ki attack inputunu alabilelim ve bu metodu oraya taşıyabilelim.
    #self.player = Player((2000,1430),[self.visible_sprites],self.obstacle_sprites,self.create_attack,self.destroy_attack,self.create_magic) # Metodu çalıştırmıyoruz sadece çağırıyoruz.
    # Player nesnesinin tüm parametrelerini verdik. create_attack, destroy_attack ve create_magic metotlarına Level içerisinden de ulaşabilmek için Player'ın bir parametresi olarak veriyoruz.

  def create_attack(self): # Bu metot ile player içerisinde oluşturduğumuz attack inputu'nu burada da alabileceğiz. Level'da bir weapon oluşsun ki düşmanla etkileşime girebilsin.
    self.current_attack = Weapon(self.player,[self.visible_sprites,self.attack_sprites]) # Weapon nesnemizi oluşturuyoruz. Her bir create_attack metodu çalıştığında. 2 parametresi var ve ilki player ikincisi ise group.
    # Ve current_attack değişkenine veriyoruz ki ekranda bir silah sprite'ı var mı yok mu kontrol edebilelim.
    # self.player'ı create_map içerisinde oluşturmuştuk bu şekilde çağırabiliriz.
    # Ve görünür olması için şu anlık sadece visible_sprites grubuna dahil ediyoruz.
    # create_attack metodunu player nesnesine bir parametre olarak veriyoruz ki player içerisinde kullanabilelim.
    # Silah sprite'ı oluştuğunda onu aynı zamanda self.attack_sprites içerisine koyacağım.

  def create_magic(self,style,strength,cost): # Bizim için büyü oluşturacak metodu yazıyoruz.
    if style == 'heal': # Eğer heal büyüsünü kullanıyorsak.
      self.magic_player.heal(self.player,strength,cost,[self.visible_sprites]) # MagicPlayer class'ından oluşturduğumuz magic_player nesnesi üzerinden heal metodunu çağırıyoruz.
      # paramatre olarak player'ı strength ve cost'u vereceğiz.

    if style == 'flame': # Eğer flame büyüsünü kullanıyorsak.
      self.magic_player.flame(self.player,cost,[self.visible_sprites,self.attack_sprites])
      # player'ı vereceğiz.
      # cost ise costu olacak
      # visible_sprites da grubu olacak fakat sonradan attack_sprites'ı da ekleyeceğiz. EKLEDİK.

    # style hangi tarzda bir büyü olduğunu, strength büyü gücünü ve cost ise maliyetini tutacak.

  def destroy_attack(self): # Oluşturduğumuz silah sprite'larını ekrandan tekrar silecek olan metodu yazıyoruz.
    if self.current_attack: # Eğer ekranda bir silah sprite'ı varsa silebilsin diye kontrol ediyoruz.
      self.current_attack.kill() # Ve sprite'ı ekrandan siliyoruz.
    self.current_attack = None # En son olarak da tekrar değişkenimizi None yapıyoruz.

  def player_attack_logic(self): # bu metot ile attack_sprites içerisinin tamamını tek tek döneceğiz ve herhangi bir attackable_sprites ile collide yapmış mı ona bakacağız.
    if self.attack_sprites: # Eğer bir attack_sprites'ı varsa ekranda
      for attack_sprite in self.attack_sprites: # attack_sprites içerisindeki her bir attack_sprite için
        collision_sprites = pygame.sprite.spritecollide(attack_sprite,self.attackable_sprites,False) # pygame.sprite.spritecollide(sprite,group,DOKILL) parametrelerini alır. Son parametre dokunulan nesneyi yok eder.
        # self.attackable_sprites içerisindeki herhangi bir sprite ile collide yapıyor mu bakıyoruz. (spritecollide grup içerisindeki her bir sprite için kontrol ediyor.)
        # Bu komut bize bir liste döndürecek.
        if collision_sprites: # Eğer herhangi bir collision varsa.
          for target_sprite in collision_sprites: # Silahımızın temas ettiği her bir nesne için
            # Eğer sprite_type'ı bir çimen ise o yok olmalı.
            # Eğer bir düşman ise canı azalmalı.
            # target_sprite.kill() # Denemek için temas ettiğimiz tüm nesneleri yok ediyoruz.
            if target_sprite.sprite_type == 'grass': # her bir sprite için kendi oluşturduğumuz sprite_type değişkenine bakıyoruz.
               # Şimdi bu kısma eğer temas varsa çimen yok olma efekti ekleyeceğiz.
               pos = target_sprite.rect.center # Hedef sprite'ımızın konumunu alıyoruz ki oraya ekleyelim efekti.
               offset = pygame.math.Vector2(0,75) # Offset ekleyerek daha gerçekçi yapıyoruz.
               for leaf in range(randint(3,6)): # Her bir yok olma animasyonu için içerisinden 3-6 arasında efekt çağırıyorum ki gerçekçi olsun.
                self.animation_player.create_grass_particles(pos - offset,[self.visible_sprites])
               # Bu metot pos ve groups parametrelerini alıyor.
               target_sprite.kill() # Çimeni yok ediyoruz.
            else:
              target_sprite.get_damage(self.player,attack_sprite.sprite_type) # Düşmanın hasar almasını sağlayan metodu çağıracağız.
            # self.player parametresini player'ın ne yaptığını öğrenmek için aldık.
            # attack_sprite.sprite_type ile büyü mü yoksa silah mı kullandığımıza bakacağız.
            # get_damage metodunu Enemy içerisinde oluşturacağız.

  def damage_player(self,amount,attack_type): # Bu metot ile düşmanlar player'a hasar verebilecek. Bu metodu canavarları Level içerisinde oluşturuken bir parametre olarak vereceğim.
  # amount hasar miktarını alacak ve attack_type ise monster_data içerisinden saldırı türünü alacak ve ona göre partikül ekleyeceğiz.
    if self.player.vulnerable: # Eğer karakterimiz hasar alabilecek durumda ise.
      self.player.health -= amount # Amount kadar player'ın canını azaltıyoruz.
      self.player.vulnerable = False # Ve karakterimizin tekrar hasar alabilmesi için bir timer oluşturmak için hasar almayı kapatıyoruz. Timer ile bunu tekrar True yapacağız.
      self.player.hurt_time = pygame.time.get_ticks() # Timer için gereken zamanlayıcıyı oluşturacak olan zaman tutucu.
      
      # Partikül efektleri
      self.animation_player.create_particles(attack_type,self.player.rect.center,[self.visible_sprites]) # Partikülleri oluşturacak olan metot.
      # Karakter hasar alırken ne tür bir hasar almış ona bakacağız ve ona göre animasyon oluşturacağız. Attack_type'ı yukarıdaki metot içerisinden alıyoruz.
      # Animasyonu nerede oluşturacağım onun için pos
      # Ve hangi grupta olacak onun için de groups
      # Bu metodu particles içerisinde oluşturacağız.

  def trigger_death_particles(self,pos,particle_type): # Düşman öldüğünde animasyonları tetikleyecek olan metot.
    self.animation_player.create_particles(particle_type,pos,self.visible_sprites) # create_particles(animation_type,pos,groups)
    # animasyon türü için particle type'ı alacağım.
    # pos için pos alacağım
    # grup için ise görünür sprite'ları kullanacağım.
    # Bu metodu enemy oluşumunda parametre olarak vereceğim ki Level'da oluşturduğum metotu enemy içerisinde de kullanabileyim.

  def add_exp(self,amount): # Player'ın her düşman öldürdüğünde ondan xp alacağı metot.
    # Bu metodu Enemy içerisinde çağıracağım. Bu nedenle Level içeirisinde Enemy oluştururken parametre olarak vereceğim.
    self.player.exp += amount # Verilen miktar kadar karakterin exp'sini arttır.

  def toggle_menu(self): # Oyunda m tuşuna basıldığında menüyü açacak olan metot.
    self.game_paused = not self.game_paused # Oyunu bir başlatıp bir durdurmamızı sağlayacak.

  def run(self): # Oyunu güncelleyip çizecek olan metot.
    self.visible_sprites.custom_draw(self.player) # Kendi oluşturduğumuz çizme metodu ile çizdiriyoruz. Ve parametre olarak self.player'ı veriyoruz. Bunu her zaman çizeceğimiz için bloklar içerisine koymadık.
    # Görünecek olan spriteların bulunduğu grubu çiziyoruz.
    self.ui.display(self.player) # UI class'ı üzerinden oluşturduğumuz self.ui üzerinden display metodu aracılığıyla verdiğim player parametresinin statlarını gösteren bir arayüz oluşturacağız. Bunu her zaman çizeceğimiz için bloklar içerisine koymadık.

    if self.game_paused: # Eğer oyun durmuşsa
      self.upgrade.display() # Ekrana upgrade menüsünü oluşturan metodu çağırdım. (upgrade nesnesi üzerinden.)
      # Geliştirme ekranını açacağım
    else: # Eğer oyun devam ediyorsa.
      # Oyunu devam ettiriyoruz.
      self.visible_sprites.update() # update metodu ile spriteların hareketlerini kontrol edebiliriz. 
      self.visible_sprites.enemy_update(self.player) # Enemy class'ı için gerekli olan self.player parametresini veriyor ve düşman durumlarını güncelleyen metotu çağırıyorum. Fakat oyun devam ediyorsa hareketlerini güncelleyeceğiz. 
      self.player_attack_logic() # Silah ve düşman ya da nesne etkileşimine bakan metodu çağırıyoruz.

class YSortCameraGroup(pygame.sprite.Group): # Kendim bir sınıf oluşturdum ve bunun bir grup olduğunu söyledim
  def __init__(self):

    # genel ayarlar
    super().__init__()
    self.display_surface = pygame.display.get_surface() # Burası içerisine yeni bir display ekranı oluşturduk.

    self.half_width = self.display_surface.get_size()[0] // 2 # Ekranın genişliğinin yarısını alıyoruz.
    self.half_height = self.display_surface.get_size()[1] // 2 # Ekranın uzunluğunun yarısını alıyoruz.

    self.offset = pygame.math.Vector2() # Kamera için  kaymayı tutacağımız bir vektör oluşturduk. Default'u (0,0) verir.
  
    # zeminin oluşturulması (fotoğraftan oluşacak ve her zaman en altta olacak bu nedenle for döngüsünün üstüne yazdık.)
    self.floor_surf = pygame.image.load('graphics/tilemap/ground.png').convert() # Zeminin fotoğrafıonı yükledik.
    self.floor_rect = self.floor_surf.get_rect(topleft = (0,0)) # Fotoğrafı verilen rect'ini oluşturduk.

  def custom_draw(self,player): # Kendi yaptığımız çizim metodunu yazıyoruz. Çünkü kamera için hepsini tek tek tekrar çizdireceğiz.
    # player parametresi kamera için oyuncunun konumuna ulaşmamız için gerekli

    # kayma miktarını getirmek
    self.offset.x = player.rect.centerx - self.half_width # x eksenindeki kayma miktarı için oyuncunun rect'inin x koordinatından sayfa genişliğinin yarısını çıkarıyoruz.
    self.offset.y = player.rect.centery - self.half_height # y eksenindeki kayma miktarı için oyuncunun rect'inin y koordinatından sayfa uzunluğunun yarısını çıkarıyoruz.

    # zemini çizme
    floor_offset_pos = self.floor_rect.topleft - self.offset # kamera için zeminin kayma miktarını aldık.
    self.display_surface.blit(self.floor_surf,floor_offset_pos) # Display ekranımıza verilen resmi kayma miktarıyla çiziyoruz.

    # 1- 
       # for sprite in self.sprites(): # Gruptaki tüm sprite'lar için
    # 2- Y pozisyonlarına göre sıralama yapacağız çünkü karakterden sonra çizilenler karakterin önünde kalıyor.
    for sprite in sorted(self.sprites(),key = lambda sprite: sprite.rect.centery): # Tüm spriteların y konumlarını alıyoruz ve buna göre sorted metodunda sıralama yapılacağını söylüyoruz.(self.sprites()(gruptaki spritelar) üzerinde yapılacak bu işlem)
      # Y posizyonlarına göre çizdiriyoruz. Yani her çizildiğinde konunları alıp sıralıyoruz bu şekilde overlap'i sağladık.
      offset_pos = sprite.rect.topleft - self.offset # Her bir sprite'ın sol üst konumuna belirtmiş olduğumuz kaymayı çıkararak yeni bir değişkene atadık.
       # self.display_surface.blit(sprite.image,sprite_rect) # Display ekranınıa her bir sprite'ın image ve rect'ini çiziyoruz.
      self.display_surface.blit(sprite.image,offset_pos) # Aldığımız değerleri verilen konumda çizdiriyoruz.

  def enemy_update(self,player): # Her bir düşmanı güncelleyecek metot. Ayrıca player parametresini de verebiliyorum. Bunu run metodu içerisinde çağıracağım.
    enemy_sprites = [sprite for sprite in self.sprites() if hasattr(sprite,'sprite_type') and sprite.sprite_type == 'enemy'] # Tüm düşman sprite'larını içeren liste
    # Tüm spriteslar arasından sadece enemyleri alacağım. Mesela Tile class'ı ve Enemy class'ı kendine ait bir sprite_type'ı var.
    # hasattr metodu önce neye bakacağını sonra da hangi attr'a bakacağına bakar. Eğer varsa ve enemy ise al ve listeye ekle diyoruz.
    for enemy in enemy_sprites: # Her bir düşman için
      enemy.enemy_update(player) # Düşman hareketlerini ve durumunu kontrol eden metodu çalıştır.

In [ ]:
# Tile class'ı ile oyundaki ağaç taş gibi nesneleri oluşturacağız
# Oyuna gerçekçilik katmak için overlap(üst üste gelme) yapmamız gerekiyor. Bunun için 2 farklı alan oluşturacağız.
# İlk alan direkt bizim full rect'imiz, ikincisi ise hitbox alanımız olacak. Hitboxları y ekseninden küçülterek üst üste binmeyi sağlayabiliriz.

import pygame 
from settings import *

# 1- Tile class'ının eski hali
"""
class Tile(pygame.sprite.Sprite): # Tile class'ı bir sprite olacağı için ordan kalıtım aldık.
  def __init__(self,pos,groups): # Ayrıca pozisyon parametresi alıyoruz nereye yerleştireceğimizi bilmek için
    # groups ile Tile objesinin hangi grupta olduğuna bakabiliriz.
    super().__init__(groups) # Sprite class'ını da initialize ediyoruz. 
    self.image = pygame.image.load('graphics/test/rock.png').convert_alpha() # Ekrana çizeceğimiz Tile objesinin resmi için bir değişken oluşturuyor ve bir fotoğraf yüklüyoruz.
    self.rect = self.image.get_rect(topleft = pos) # Ekrana çizeceğimiz Tile objesinin bir temas alanını yaratıyoruz. # Bu bizim çizim için kullandığımız alan olacak.
    # Aldığımız position ile de konumunu ayarlıyoruz.
    self.hitbox = self.rect.inflate(0,-10) # inflate metodu verilen değerler kadar rect'i küçültür. Bu örnekte -5 birim yukarı -5 birim aşağıdan kısaltarak x eksenine dokunmayacağız.
"""
# 2- Tile classını daha esnek hale getiriyoruz.

class Tile(pygame.sprite.Sprite): # Tile class'ı bir sprite olacağı için ordan kalıtım aldık.
  def __init__(self,pos,groups,sprite_type,surface = pygame.Surface((TILESIZE,TILESIZE))): # Ayrıca pozisyon parametresi alıyoruz nereye yerleştireceğimizi bilmek için
    # groups ile Tile objesinin hangi grupta olduğuna bakabiliriz.
    # sprite type ile hangi türden bir sprite olduğunu kontrol edeceğiz.
    # surface eğer herhangi bir yüzey yüklenmemişse bizim için siyah bir kare oluşturacak. Eğer yüklenmişse o png gösterilecek.
    super().__init__(groups) # Sprite class'ını da initialize ediyoruz. 
    self.sprite_type = sprite_type # Sprite türümüzün değişkenini oluşturduk.

    y_offset = HITBOX_OFFSET[sprite_type] # sprite_type'ına göre bir hitbox değeri alıyoruz.

    self.image = surface # Ekrana çizeceğimiz Tile objesinin resmi için bir değişken oluşturuyor ve bir fotoğraf yüklüyoruz. Parametreden gelen fotoyu koyacağız.
    
    if sprite_type == 'object': # Eğer sprite türümüz bir obje ise:
      # Bir offset yapmalıyız:
      self.rect = self.image.get_rect(topleft = (pos[0],pos[1] - TILESIZE)) # X ekseninde bir değişim yapmıyoruz fakat y eksenini TILESIZE kadar yukarı kaydırıyoruz ki fotoğrafın en köşesine gelsin.
    else: # Eğer grass ise bir sorun yok aynen çizdirebiliriz.
      self.rect = self.image.get_rect(topleft = pos) # Ekrana çizeceğimiz Tile objesinin bir temas alanını yaratıyoruz. # Bu bizim çizim için kullandığımız alan olacak.
    # Aldığımız position ile de konumunu ayarlıyoruz.
    
    self.hitbox = self.rect.inflate(0,y_offset) # inflate metodu verilen değerler kadar rect'i küçültür. Bu örnekte settings içerisinden alacağımız değere göre bir offset veriyoruz.

In [ ]:
# Player class'ı ile oyundaki karakterimizi oluşturacağız.
# Oyuna gerçekçilik katmak için overlap(üst üste gelme) yapmamız gerekiyor. Bunun için 2 farklı alan oluşturacağız.
# İlk alan direkt bizim full rect'imiz, ikincisi ise hitbox alanımız olacak. Hitboxları y ekseninden küçülterek üst üste binmeyi sağlayabiliriz.

import pygame 
from settings import *
from support import *
from entity import Entity

class Player(Entity): # (1)Player sınıfı bir sprite olacak ve ordan kalıtım alacak. (2) Player sınıfı Entity sınıfından kalıtım alacak ve zaten Entity sınıfı da sprite'lardan kalıtım alıyor.
  def __init__(self,pos,groups,obstacle_sprites,create_attack,destroy_attack,create_magic): # Ayrıca pozisyon parametresi alıyoruz nereye yerleştireceğimizi bilmek için
    # groups ile Player objesinin hangi grupta olduğuna bakabiliriz.
    # obstacle_sprites ile karakterin haritadaki nesnelerden haberi olmasını ve çarpışmasını sağlıyoruz.
    # create_attack parametresi Level içerisindeki create_attack metoduna ulaşabilmemiz için gerekli. Eğer saldırı inputu verilmişse alacağız ve kullanacağız.
    # create_magic parametresi Level içerisindeki create_magic metoduna ulaşabilmemiz için gerekli. Eğer büyü inputu verilmişse alacağız ve kullanacağız.
    super().__init__(groups) # Sprite class'ını da initialize ediyoruz. 
    self.image = pygame.image.load('graphics/test/player.png').convert_alpha() # Ekrana çizeceğimiz Player objesinin resmi için bir değişken oluşturuyor ve bir fotoğraf yüklüyoruz.
    self.rect = self.image.get_rect(topleft = pos) # Ekrana çizeceğimiz Player objesinin bir temas alanını yaratıyoruz.
    # Aldığımız position ile de konumunu ayarlıyoruz.
    self.hitbox = self.rect.inflate(-6,HITBOX_OFFSET['player']) # Bu da overlap yapabilmemiz için gerekli olan hitbox'ımız.
    # Hitbox değerini settings'den aldık.

    # Grafik ayarları (animasyonlar)
    self.import_player_assets() # Karakter animasyonlarını yükleyen metodu init kısmında karakter çağırılken çalıştırıyoruz.
    self.status = 'down' # Karakterimizin hangi durumda olduğunu bu değişken ile belirleyerek ona göre animasyona ulaşıp animasyonu gerçekleştireceğiz. default olarak down'ı verdik.
    
    """ Bu değişkenleri entity sınıfının içerisinde tanımladık.
    self.frame_index = 0 # Karakter animasyonlarını içeren durum listeleri içerisinde sırayla fotoğrafları indexlerine göre döneceğimiz için bir index oluşturuyoruz.
    self.animation_speed = 0.15 # Karakter animasyonlarını içeren surface listesi içerisinde yukarıdaki index'imizi artıracak olan animasyon hızımızı belirliyoruz.
    """

    # Hareket için gerekliler:
    """ Bu değişkeni de Entity içerisinde tanımladık.
    self.direction = pygame.math.Vector2() # Karakterimizin yöneldiği yönü göstermek için bir değişken oluşturduk
    """
    # Bu değişken (x,y) şeklinde ve eğer sağa gidiyorsak 1,0, sola gidiyorsak -1,0
    # yukarı gidiyorsak 0,-1 aşağı gidiyorsak 0,1 değerlerini alacak.
     # self.speed = 5 # Karakterimizin yürümesi için bir hız değişkeni oluşturduk. Stats bölümünde yenisi var.
    self.attacking = False # Saldırıyor olma durumumuzun kontrolü için bir değişken oluşturduk.
    self.attack_cooldown = 400 # Tekrar saldırma süremiz için bir değişken oluşturduk.
    self.attack_time = None # Saldırı süremiz için bir değişken oluşturduk. Bu değişkeni her salırdığımızdan itibaren tutmaya başlayacağız.

    # Silahlar
    self.create_attack = create_attack # Level class'ı içerisinde oluşturduğumuz create_attack metodunu burada da kullanabilmek için tanımlıyoruz.
    self.destroy_attack = destroy_attack # Level class'ı içerisinde oluşturduğumuz destroy_attack metodunu burada da kullanabilmek için tanımlıyoruz.
    # destroy_attack metodunu cooldowns içerisinde çağırmalıyım. Çünkü saldırı bittikten sonra yani o cooldowndan sonra silahı ekrandan silmeli.
    self.weapon_index = 0 # Silah seçimi için bir index değeri oluşturuyoruz.
    # Oyun içerisinde input alarak silah değişimini yapacağız.
    self.weapon = list(weapon_data.keys())[self.weapon_index] # Weapon_data bizim silahlarımızı tutan dictionary ve keys kısmında silahlarımızın isimleri var. Dosya isimleri de aynı olduğu için kolaylıkla ulaşabiliriz.
    # weapon_index'imize göre bu keys yani silahlardan birini seçeceğiz. Bu nedenle bir listeye çeviriyoruz.
    self.can_switch_weapon = True # Silahı değiştirip değiştiremeyeceğimizi söyleyen değişken
    self.weapon_switch_time = None # Silah değiştirme süremiz için bir değişken oluşturduk. Bu değişken ile timer oluşturacağız.
    self.switch_duration_cooldown = 200 # Tekrar silah değiştirebilmemiz için gereken süre.

    # Büyüler
    self.create_magic = create_magic # Level class'ı içerisinde oluşturduğumuz create_magic metodunu burada da kullanabilmek için tanımlıyoruz.
    self.magic_index = 0 # Büyü seçimi için gerekli olan index değeri
    # Oyun içerisinde input alarak büyü değişimini yapacağız.
    self.magic = list(magic_data.keys())[self.magic_index] # magic_data bizim büyülerimizi tutan dictionary ve keys kısmında büyülerimizin isimleri var. Dosya isimleri de aynı olduğu için kolaylıkla ulaşabiliriz.
    # magic_index'imize göre bu keys yani büyülerden birini seçeceğiz. Bu nedenle bir listeye çeviriyoruz.
    self.can_switch_magic = True # Büyüyü değiştirip değiştiremeyeceğimizi söyleyen değişken
    self.magic_switch_time = None # Büyü değiştirme süremiz için bir değişken oluşturduk. Bu değişken ile timer oluşturacağız.
     # self.switch_duration_cooldown = 200 # Tekrar büyü değiştirebilmemiz için gereken süre. Silahla aynı old için gerek yok.

    # Statlar
    self.stats = {'health':100, 'energy':60, 'attack':10, 'magic':4, 'speed':6} # Karakterimizin statlarını oluşturduk.
    self.max_stats = {'health':300, 'energy':140, 'attack':20, 'magic':10, 'speed':10} # Karakterimizin tecrübelerle geliştirebileceği maksimum stat'lar
    self.upgrade_cost = {'health':100, 'energy':100, 'attack':100, 'magic':100, 'speed':100} # Her bir geliştirme için gereken tecrübe puanı miktarı.
    self.health = self.stats['health'] # Bir can değişkeni oluşturduk.
    self.energy = self.stats['energy'] # Bir enerji değişkeni oluşturduk.
    self.exp = 500 # UI için bir enerji oluşturduk.
    self.speed = self.stats['speed'] # Hız değişkenimizi oluşturduk. Eskisini silebiliriz.
    # Oyun içerisinde değişebilecek değişkenlerimizi ayrı bir değişkende tutuyoruz.

    # Hasar alabilme zamanlayıcısı
    self.vulnerable = True # Hasar alabilme
    self.hurt_time = None # Timer için gerekli zamanlayıcı
    self.invulnerability_duration = 500 # Tekrar hasar alabilmesi için geçmesi gereken süre

    self.obstacle_sprites = obstacle_sprites # obstacle_sprites grubunu player için de tanımlıyoruz.

    # Sesler
    self.weapon_attack_sound = pygame.mixer.Sound('audio/sword.wav') # saldırı sesimiz
    self.weapon_attack_sound.set_volume(0.4) # Saldırı ses düzeyimiz

  def import_player_assets(self): # Karakterin tüm animasyonlarını import edeceğiz.
    character_path = 'graphics/player/' # Tüm animasyonların temelinde bulunan dosya yolunu bir değişkene atıyoruz.

    self.animations = { # Karakterimizin 12 farklı animasyonu için bir dictionary oluşturduk.
        'up': [], 'down':[],'left':[],'right':[], # Yürüyüş animasyonları
		    'right_idle':[],'left_idle':[],'up_idle':[],'down_idle':[], # Boş durma animasyonları her yön için
		    'right_attack':[],'left_attack':[],'up_attack':[],'down_attack':[] # Her yön için saldırı animasyonları
    }
    # Karakter animasyonlarımızın bulunduğu dosyaların isimleri yukarıdaki dictionary'lerin key isimleri ile aynı. Bu şekilde kolayca ulaşabileceğiz.

    for animation in self.animations.keys(): # Verilen dictionary'nin tüm keyslerini animasyon olarak bakıyoruz.
      # Burada tüm animasyonlar için gerekli aynı isimli dosyaya ulaşacağım. (Aşağıda tam dosya yolunu oluşturduktan sonra.)
      full_path = character_path + animation # Bu bize 'graphics/player/up' gibi bir şey döndürecek.
      # Sıradaki işlem çok önemli. Yukarıdaki dosya yoluna ulaştıktan sonra önceden oluşturduğumuz ve tüm png'leri bir surface olarak bize veren metodu kullanarak dictionary içerisindeki values boşluklarını elde ettiğimiz surface'lar ile dolduracağız.
      self.animations[animation] = import_folder(full_path) # animasyonlar dictionary'si içerisindeki örneğin 'up' key'inin value'suna metot ile getirmiş olduğumuz surface'ları veriyoruz ve bunu tüm animasyonlar için yapıyoruz. 
    # Çıktı bize her animasyon için surface'lardan oluşan bir liste döndürecek.

  def input(self): # Klavyeden girilen inputları alan fonksiyon
    keys = pygame.key.get_pressed() # Klavyedeki tüm tuşları aldık.

    if not self.attacking: # Eğer saldırı yapıyor isek hiçbir klavye input'u almak istemiyorum.
      # Hareket
      if keys[pygame.K_UP]: # Eğer klavyede yukarı ok tuşuna basıyorsak
        self.status = 'up' # Karakterimiz yukarı yönde gidiyor olduğu için durumunu 'up' yapıyoruz ki animasyonuna kolayca ulaşabilelim.
        self.direction.y = -1 # Player direction'ının y'sini -1 yap.
      elif keys[pygame.K_DOWN]: # Eğer klavyede aşağı ok tuşuna basıyorsak
        self.status = 'down' # Karakterimiz aşağı yönde gidiyor olduğu için durumunu 'down' yapıyoruz ki animasyonuna kolayca ulaşabilelim.
        self.direction.y = 1 # Player direction'ının y'sini 1 yap.
      else: # Eğer bu tuşlara basılmıyorsa direction'ın y'si 0 olsun.
        self.direction.y = 0

      if keys[pygame.K_RIGHT]: # Eğer klavyede sağ ok tuşuna basıyorsak
        self.status = 'right' # Karakterimiz sağ yönde gidiyor olduğu için durumunu 'right' yapıyoruz ki animasyonuna kolayca ulaşabilelim.
        self.direction.x = 1 # Player direction'ının x'sini 1 yap.
      elif keys[pygame.K_LEFT]: # Eğer klavyede sol ok tuşuna basıyorsak
        self.status = 'left' # Karakterimiz sol yönde gidiyor olduğu için durumunu 'left' yapıyoruz ki animasyonuna kolayca ulaşabilelim.
        self.direction.x = -1 # Player direction'ının x'sini -1 yap.
      else: # Eğer bu tuşlara basılmıyorsa direction'ın x'si 0 olsun.
        self.direction.x = 0

      # Saldırı
      if keys[pygame.K_SPACE]: # Eğer space tuşuna basıyorsak (ve saldırma animasyonu içerisinde değilsek) saldır. Bu şekilde art arda saldırı yapmayı önlüyoruz. Fakat bastığımızda pygame birden çok kez tetikleniyor. Bu yüzden bir timer oluşturmamız lazım.
        self.attacking = True # Değişkenimizi True yapıyoruz çünkü bir saldırı işlemi içerisindeyiz.
        self.attack_time = pygame.time.get_ticks() # Oluşturduğum değişken ise sadece space'e basılınca tek seferlik zamanı hesaplıyor.
        self.create_attack() # Level class'ı içerisinden alıp burada çağırdığımız metodu çalıştırıyoruz her bir attack işleminde.
        self.weapon_attack_sound.play() # Ve her saldırıda bu sesi çalıyoruz.

      # Büyü
      if keys[pygame.K_LCTRL]: # Eğer sol ctrl'ye basıyorsak (ve saldırma animasyonu içerisinde değilsek) büyü kullan. Fakat bastığımızda pygame birden çok kez tetikleniyor. Bu yüzden bir timer oluşturmamız lazım.
        self.attacking = True # Değişkenimizi True yapıyoruz çünkü bir saldırı işlemi içerisindeyiz.
        self.attack_time = pygame.time.get_ticks() # Oluşturduğum değişken ise sadece LCTRL'e basılınca tek seferlik zamanı hesaplıyor.
        style = list(magic_data.keys())[self.magic_index] # Büyünün türü
        strength = list(magic_data.values())[self.magic_index]['strength'] + self.stats['magic'] # Büyünün gücü (Karakterin kendinde olan büyü hasarını da ekliyoruz.)
        cost = list(magic_data.values())[self.magic_index]['cost'] # Büyü maliyeti
        self.create_magic(style,strength,cost) # Create_magic için style,strength ve cost lazım. Kullanıcı her büyü kullandığında LCTRL ile Level class'ı içerisindeki bu metot çalışacak.
      
      # Silah değişimi
      if keys[pygame.K_q] and self.can_switch_weapon: # Eğer oyun içerisinde q tuşuna basarsak ve silah değiştirebiliyorsak(değişken True ise) silah değişsin fakat bunun için de bir timer(zamanlayıcı) lazım. Sürekli silah değişmesin diye. (pygame birden fazla tetikleyebilir.)
        self.can_switch_weapon = False # Silah değiştirebilmeyi False yap.
        self.weapon_switch_time = pygame.time.get_ticks() # q'ya basıldığı gibi süreyi hesaplamaya başlayacağız ve bu süreyi değişkende tutacağız. Timer yaparken kullanacağız.
        if self.weapon_index < len(list(weapon_data.keys())) - 1: # Eğer index'imiz liste uzunluğunun 1 eksiğinden küçükse.(index hesabı)
          self.weapon_index += 1 # Silah index'ini 1 artırıyoruz.
        else: # Eğer büyükse,
          self.weapon_index = 0 # Silah index'ini tekrar 0 yapıp başa döndürüyoruz.
        # Silah index'i artıyor fakat self.weapon'ı init içerisinde aldığımız için sürekli aynı silah geliyor. Bu nedenle q ya basıldığında self.weapon'ı güncellemeliyiz.
        self.weapon = list(weapon_data.keys())[self.weapon_index] # self.weapon'ı güncelleyerek yeni silaha geçiyoruz.

      # Büyü değişimi
      if keys[pygame.K_e] and self.can_switch_magic: # Eğer oyun içerisinde e tuşuna basarsak ve büyü değiştirebiliyorsak(değişken True ise) büyü değişsin fakat bunun için de bir timer(zamanlayıcı) lazım. Sürekli büyü değişmesin diye. (pygame birden fazla tetikleyebilir.)
        self.can_switch_magic = False # Büyü değiştirebilmeyi False yap.
        self.magic_switch_time = pygame.time.get_ticks() # e'ya basıldığı gibi süreyi hesaplamaya başlayacağız ve bu süreyi değişkende tutacağız. Timer yaparken kullanacağız.
        if self.magic_index < len(list(magic_data.keys())) - 1: # Eğer index'imiz liste uzunluğunun 1 eksiğinden küçükse.(index hesabı)
          self.magic_index += 1 # Büyü index'ini 1 artırıyoruz.
        else: # Eğer büyükse,
          self.magic_index = 0 # Büyü index'ini tekrar 0 yapıp başa döndürüyoruz.
        # Büyü index'i artıyor fakat self.magic'ı init içerisinde aldığımız için sürekli aynı büyü geliyor. Bu nedenle e ya basıldığında self.magic'ı güncellemeliyiz.
        self.magic = list(magic_data.keys())[self.magic_index] # self.magic'ı güncelleyerek yeni büyüye geçiyoruz.

  def get_status(self): # Karakterimizin durumunu getirecek olan metodumuzu yazıyoruz.

    # Boş durma durumları
    if self.direction.x == 0 and self.direction.y == 0: # Karakterimiz duruyor ama en son hangi pozisyonda duruyor bunu belirlememiz lazım.
      if not 'idle' in self.status and not 'attack' in self.status: # Eğer status içerisinde idle yoksa ve attack yoksa idle ekliyoruz. Bu şekilde sürekli idle eklemiş olmayacağız ve hem saldırıp hem de boş duruyor olmayacağız.
        self.status = self.status + '_idle' # Eğer karakterimiz duruyorsa onun en son bastığımız tuşa göre yönünü alıyor ve dosya isimleri uyum sağlasın diye sonuna _idle eksliyoruz.

    # Saldırı durumları
    if self.attacking: # Saldırı yapıyor olduğumuz zaman
      self.direction.x = 0 # x'i ve y'i 0 yapıyoruz ki saldırırken hareket edemeyelim.
      self.direction.y = 0
      if not 'attack' in self.status: # Eğer status içerisinde attack yoksa attack ekliyoruz. Bu şekilde sürekli attack eklemiş olmayacağız.
        if 'idle' in self.status: # Eğer statusün içerisinde hala _idle varsa (right_idle_attack gibi)
          self.status = self.status.replace('_idle','_attack') # idle'ı kaldırıp yerine attack yazdırabiliriz.
        else: # Eğer status içerisinde _idle yoksa direkt _attack ekleyebiliriz.
          self.status = self.status + '_attack' # Eğer karakterimiz duruyorsa onun en son bastığımız tuşa göre yönünü alıyor ve dosya isimleri uyum sağlasın diye sonuna _attack eksliyoruz.
    else: # Saldırı bittiği ya da yapmadığımız zaman
      if 'attack' in self.status: # Eğer durumumuzun içerisinde attack varsa aşağıdaki işlemi yap ki hata göndermesin.
        self.status = self.status.replace('_attack','') # Durmumuzu tekrar eski haline getiriyoruz ki sürekli saldırı animasyonunda kalmayalım.

  """ Entity class'ı içerisine kopyaladık ve orada hem düşmanlar hem de player için bir çarpışma ve hareket oluşturacağız.
  #####################################################################################################
  def move(self,speed): # Karakterin yürümesini sağlayacak metodu oluşturuyoruz.
    # speed parametresi karakterin hızı olacak.

    # Fakat (1,1) gibi çift yönlü durumlarda karakter aşırı hızlı gidiyor bu sebeple bu hızı normalleştirmemiz gerek:

    if self.direction.magnitude() != 0: # Direction'ımızın bir değeri olup olmadığına bakıyoruz.(x ve y vektörleri için)
      self.direction = self.direction.normalize() # Eğer öyleyse bu hızı normalleştiriyoruz.(0.7 gibi bir değer)

     # self.rect.center += self.direction * speed # Karakterimizin rectangle'ının merkezini her saniye karakterin directionını, karakterin hızı ile çarpıp ekliyoruz.
    # Bu şekilde eğer yönümüz sola (-1,0) ise hız 5 ise (-5,0) kordinatlarına taşıyor ve hızlıca karakteri çiziyoruz. Böylece karakterimiz hareket ediyor.

    # 1- # Yönleri ayırdık çünkü hangi yönder çarpışma olduğunu bilmek istiyoruz.
    
      # self.rect.x += self.direction.x * speed # Karakterin rect'inin x konumuna direction * speed ekliyoruz.
      # self.collision('horizontal') # Yatay çarpışmanın kontrolünü yapıyoruz.
      # self.rect.y += self.direction.y * speed # Karakterin rect'inin y konumuna direction * speed ekliyoruz.
      # self.collision('vertical') # Dikey çarpışmanın kontrolünü yapıyoruz.
       
    # 2- # rect'leri hitboxlar ile değiştiriyoruz.

    self.hitbox.x += self.direction.x * speed # Karakterin hitbox'inin x konumuna direction * speed ekliyoruz.
    self.collision('horizontal') # Yatay çarpışmanın kontrolünü yapıyoruz.
    self.hitbox.y += self.direction.y * speed # Karakterin hitbox'inin y konumuna direction * speed ekliyoruz.
    self.collision('vertical') # Dikey çarpışmanın kontrolünü yapıyoruz.
    # Yönleri ayırdık çünkü hangi yönder çarpışma olduğunu bilmek istiyoruz.
    self.rect.center = self.hitbox.center # Rect'in merkeziyle hitbox'ın merkezini aynı yere koyduk ve ayarladık.

  def collision(self,direction): # Çarpışmaları gerçekleştirecek metodu yazıyoruz.
    # direction parametresi ile çarpışmanın hangi yönden olduğuna bakıyoruz.

  # 1- 
  
    # if direction == 'horizontal': # Eğer yatay eksendeysek.
    #   for sprite in self.obstacle_sprites: # obstacle_sprites grubundaki her bir sprite için
    #     if sprite.rect.colliderect(self.rect): # Eğer sprite karakterimiz ile çarpışıyorsa.
    #       if self.direction.x > 0: # Eğer karakterimizin yönü sağa doğruysa
    #         self.rect.right = sprite.rect.left # Çarpışmadan sonra karakterimizin rect'inin sağ tarafı, çarptığımız nesnenin rect'inin sol tarafının konumunda olmalı.
    #       if self.direction.x < 0: # Eğer karakterimiz sola doğruysa
    #         self.rect.left = sprite.rect.right # Çarpışmadan sonra karakterimizin rect'inin sol tarafı, çarptığımız nesnenin rect'inin sağ tarafının konumunda olmalı.
    
    # if direction == 'vertical': # Eğer dikey eksendeysek.
    #   for sprite in self.obstacle_sprites: # obstacle_sprites grubundaki her bir sprite için
    #     if sprite.rect.colliderect(self.rect): # Eğer sprite karakterimiz ile çarpışıyorsa.
    #       if self.direction.y > 0: # Eğer karakterimizin yönü aşağı doğruysa
    #         self.rect.bottom = sprite.rect.top # Çarpışmadan sonra karakterimizin rect'inin alt tarafı, çarptığımız nesnenin rect'inin üst tarafının konumunda olmalı.
    #       if self.direction.y < 0: # Eğer karakterimiz yukarı doğruysa
    #         self.rect.top = sprite.rect.bottom # Çarpışmadan sonra karakterimizin rect'inin üst tarafı, çarptığımız nesnenin rect'inin alt tarafının konumunda olmalı.
  
  # 2- rect'leri hitbox ile değiştirdik ki overlap yapabilelim.
    
    if direction == 'horizontal': # Eğer yatay eksendeysek.
      for sprite in self.obstacle_sprites: # obstacle_sprites grubundaki her bir sprite için
        if sprite.hitbox.colliderect(self.hitbox): # Eğer sprite karakterimizin hitboxı ile çarpışıyorsa.
          if self.direction.x > 0: # Eğer karakterimizin yönü sağa doğruysa
            self.hitbox.right = sprite.hitbox.left # Çarpışmadan sonra karakterimizin hitbox'ının sağ tarafı, çarptığımız nesnenin hitbox'inin sol tarafının konumunda olmalı.
          if self.direction.x < 0: # Eğer karakterimiz sola doğruysa
            self.hitbox.left = sprite.hitbox.right # Çarpışmadan sonra karakterimizin hitbox'inin sol tarafı, çarptığımız nesnenin hitbox'inin sağ tarafının konumunda olmalı.
    
    if direction == 'vertical': # Eğer dikey eksendeysek.
      for sprite in self.obstacle_sprites: # obstacle_sprites grubundaki her bir sprite için
        if sprite.hitbox.colliderect(self.hitbox): # Eğer sprite karakterimiz ile çarpışıyorsa.
          if self.direction.y > 0: # Eğer karakterimizin yönü aşağı doğruysa
            self.hitbox.bottom = sprite.hitbox.top # Çarpışmadan sonra karakterimizin hitbox'inin alt tarafı, çarptığımız nesnenin hitbox'inin üst tarafının konumunda olmalı.
          if self.direction.y < 0: # Eğer karakterimiz yukarı doğruysa
            self.hitbox.top = sprite.hitbox.bottom # Çarpışmadan sonra karakterimizin hitbox'inin üst tarafı, çarptığımız nesnenin hitbox'inin alt tarafının konumunda olmalı.
  #####################################################################################################
  """
  def cooldowns(self): # Bu metot ile timer tutmamız gereken hareketler için bir cooldown ooluşturuyoruz.
    current_time = pygame.time.get_ticks() # Anlık zamanı sürekli olarak hesaplıyoruz.

    # Saldırı Zamanlayıcısı
    if self.attacking: # Eğer saldırı yapıyorsak aşağıdaki işlemleri yap.
      if current_time - self.attack_time >= self.attack_cooldown + weapon_data[self.weapon]['cooldown']: # Eğer sürekli hesaplanan oyun zamanından saldırı süremizi çıkarırsak ve bu da saldırı cooldown'ından fazla veya eşit ise (silah ve karakter cooldown'ının toplamı)
        self.attacking = False # Saldırı bitmiş demektir ve tekrar saldırı yapabilmemiz için self.attacking'i False yapıyoruz.
        self.destroy_attack() # Saldırı cooldown'ı bittikten sonra silahı ekrandan siliyoruz.

    # Silah Zamanlayıcısı
    if not self.can_switch_weapon: # Silah değiştirirken bu değişken False oluyor bu nedenle not kullanıyoruz. Yani silah değişirken,
      if current_time - self.weapon_switch_time >= self.switch_duration_cooldown: # Eğer sürekli hesaplanan oyun zamanından q'ya bastığımızda hesaplanan zamanı çıkarırsak ve bu da silah değiştirme cooldown'ından fazla veya eşit ise
        self.can_switch_weapon = True # Silahı değiştirmiş demektir ve tekrar silah değiştirebilmemiz için self.can_switch_weapon'ı True yapıyoruz.

    # Büyü Zamanlayıcısı
    if not self.can_switch_magic: # Büyü değiştirirken bu değişken False oluyor bu nedenle not kullanıyoruz. Yani büyü değişirken,
      if current_time - self.magic_switch_time >= self.switch_duration_cooldown: # Eğer sürekli hesaplanan oyun zamanından e'ya bastığımızda hesaplanan zamanı çıkarırsak ve bu da büyü değiştirme cooldown'ından fazla veya eşit ise
        self.can_switch_magic = True # Büyü değiştirmiş demektir ve tekrar büyü değiştirebilmemiz için self.can_switch_magic'ı True yapıyoruz.

    # Tekrar hasar alabilme zamanlayıcısı
    if not self.vulnerable: # Eğer hasar almışsak
      if current_time - self.hurt_time >= self.invulnerability_duration: # Cooldown sağlanıyorsa
        self.vulnerable = True # Hasar alabilmeyi tekrardan True yap.

  def animate(self): # Bu metot ile oyuncunun status'üne göre animasyon fotoğraflarını tek tek gezen bir döngü oluşturacağız.
    animation = self.animations[self.status] # Animasyonumu öğrenmek için animations dictionary'm içerisindeki self.status durumuna gideceğim ve o da bize yardımcı metodumuz ile doldurmuş olduğumuz surface listelerini getirecek.

    # Animasyon döngü işlemleri
    self.frame_index += self.animation_speed # Sürekli index'leri döndürecek olan index'i animasyon hızımızı sürekli ekleyerek döndürüyoruz.
    if self.frame_index >= len(animation) -1: # Eğer index'imiz liste içerisindeki animasyon png'lerinin index numarasının üstüne çıkarsa,
      self.frame_index = 0 # index'imizi tekrar 0 yapıyoruz ki sürekli dönsün.

    # Animasyonların ayarlanması
    self.image = animation[int(self.frame_index)] # Karakterin fotoğrafını doğru animasyon png'lerinin içinde bulunduğu listeden index yardımı ile seçiyoruz. Integer'a çevirmemiz gerekiyor ki tam sayı alalım.
    self.rect = self.image.get_rect(center = self.hitbox.center) # Her hareketin ardına da düzenin bozulmamsı için yeni bir rect oluşturuyoruz.

    # Player'ın hasar alması. Bunun için Level içerisinde birkaç tane metot oluşturacağız. Ve hasar vermeyi Enemy'e vereceğiz.
    # Player'ın hasar alınca titremesi

    if not self.vulnerable: # Eğer karakter hasar almışsa
      alpha = self.wave_value() # Entity içerisindeki 255 ya da 0 transparanlık değeri döndüren metot ile değeri alıyoruz.
      self.image.set_alpha(alpha)
    else: # Eğer hasar almamışsa
      self.image.set_alpha(255)

  def get_full_weapon_damage(self): # Toplam silah hasarını getirecek metod. Canavarın canını azaltacak miktar.
    base_damage = self.stats['attack'] # Karakterin temel hasarı
    weapon_damage = weapon_data[self.weapon]['damage'] # Silahın hasarı
    return base_damage + weapon_damage # Karakter ve silahın toplam hasarı
  
  def get_full_magic_damage(self):
    base_damage = self.stats['magic'] # Karakterin temel büyü hasarı
    spell_damage = magic_data[self.magic]['strength'] # Büyü hasarı
    return base_damage + spell_damage # Karakter ve büyünün toplam hasarı

  def get_value_by_index(self,index): # Indeks değeri ile stat değerlerini getiren metot.
    return list(self.stats.values())[index] # Stat değerleri içerisinden index ile değeri getiriyoruz.

  def get_cost_by_index(self,index): # Indeks değeri ile stat maliyetlerini getiren metot.
    return list(self.upgrade_cost.values())[index] # upgrade_cost dict içerisinden değerleri alıyor ve index ile seçiyorum.

  def energy_recovery(self): # Oyun içinde enerjimizi yenileyecek olan metot.
    if self.energy < self.stats['energy']: # Eğer şu anki enerjimiz maksimum enerjimizin altında ise,
      self.energy += 0.01 * self.stats['magic'] # Enerjiyi 0.01 * 4 arttır.
    else: # Olur da maks üstüne çıkarsa enerjimiz.
      self.energy = self.stats['energy'] # enerjiyi maks yap.

  def update(self): # Bu metot ile karakteri sürekli güncelleyip birden fazla metotu bir anda çağırabileceğiz.
    self.input() # Input metodunu çağırıyoruz.
    self.cooldowns() # Cooldowns metodunu çağırıp timer'ı sürekli tutturuyoruz. Böylece art arda saldırı yapmanın önüne geçebildik. Bu saldırı ve büyü için de geçerli.
    self.get_status() # Ve metodumuzu çağırarak sürekli olarak oyuncunun durumunu kontrol ediyoruz.
    self.animate() # Ve animasyonları gerçekleştirecek metodumuzu update içerisinde çağırıyoruz.
    self.move(self.stats['speed']) # move metodunu çağırıyor ve hız parametresi için stat'lar içerisinden hız'ı veriyoruz.
    self.energy_recovery() # enerjimizi yenileyen metodumuz.


In [ ]:
import pygame
# destek dosyaları için ayrı bir bölmeyi kullanıyoruz.
from csv import reader # csv modülü içerisinden reader'ı çağırıp csv dosyasını okuyacağız.
from os import walk # import_folder metodu için walk metodunu aldık.

def import_csv_layout(path): # bu metot ile csv dosyalarını işlenebilir hale getirip dünyamızı oluşturacağız.
  terrain_map = [] # bir liste oluşturuyoruz.
  with open(path) as level_map: # csv dosyasını with komutuyla açıyoruz.
    layout = reader(level_map, delimiter= ',') # reader komutuna okunacak dosyayı ve hangi yerlerden parçalanacağını belirtiyoruz.
    for row in layout: # layout içerisindeki her bir satırı yazdırıyoruz.
      terrain_map.append(list(row)) # Listeye tüm satırları liste olarak veriyoruz.
    return terrain_map # ve bu listeyi döndürüyoruz. Listelerden oluşan bir liste

# Bu işlem bize -1 ve 395 gibi sayılardan oluşmuş bir listeler bütünü verecek yukarıdaki world map gibi. -1'ler
# -1'ler bizim için boş olanları 395'ler ise bizim için bir entity'nin yerini tutacak. Yani 395'lerin içinden geçilmeyecek. Sınır alanları gibi.

def import_folder(path): # Bizim 3 farklı grass ve 20 farklı object'imiz var. Bu nedenler hepsini tek bir seferde resmini alıp bir surface a çeviren bir metot yazmamız gerekiyor.
  surface_list = []
  
  for _,__,img_files in walk(path): # walk metodu bizim için önce dosyanın konumunu sonra içindeki klasörleri bir liste şeklinde ve en sonda da içindeki dosyaları bir liste içerisinde getirir.
    # _ = dosya yolunu tutuyor gereksiz.
    # __ = alt klasörleri tutuyor gereksiz.
    # img_files = png'leri tutuyor gerekli.
    # Bu metot ile her bir fotoğrafın tam dosya konumuna ulaşıp onu pygame'e yükleyecek ve surface'a çevireceğiz.
    for image in img_files: # Her bir png için
      full_path = path + '/' + image # Her bir png için tam bir dosya yolu oluşturduk.
      image_surf = pygame.image.load(full_path).convert_alpha() # Her bir png'yi ise bir surface'a dönüştürüyoruz ve listeye atacağız.
      surface_list.append(image_surf)
  return surface_list # En son olarak da bu listeyi döndürüyoruz.

In [ ]:
# Weapons
# Her saldırı gerçekleştirdiğimizde silahımızın bir sprite'ı oluşacak ve ardından tekrar sileceğiz.
# Silahımızın sprite'ını player'ın direction'ına göre oluşturacağız.
# Silah nesnesini Level içerisinde oluşturmalıyım ki düşmanla etkileşime girebilsin. Fakat sorun şu ki biz saldır input'unu player içerisinden alıyoruz.
# Bu nedenle Level içerisine create_attack adında bir metot oluşturacağız.

import pygame

class Weapon(pygame.sprite.Sprite): # Bir silah sınıfı oluşturuyoruz ve bir sprite olacağı için Sprite sınıfından inheritance alıyoruz.
  def __init__(self,player,groups): # init ediyoruz.
    # Silahı player'ın yanında ve o yönde oluşturacağımız için player değişkeni gerekiyor.
    # Ve ayrıca groups açarak silahı çizeceğimiz için visible sprites olması gerekiyor.
    super().__init__(groups) # Sprite class'ını da init ettik.

    self.sprite_type = 'weapon' # Bir sprite_type oluşturuyoruz.

    direction = player.status.split("_")[0] # Player'ın durumunu getiriyoruz yönü ve animasyon png'sini getirebilmek için.
    # Fakat status içerisinde idle ve attack durumları da var ve bunları kullanmak istemiyorum bu nedenle split metodunu kullanıyorum.
    # Ve parçaladığım durumun sadece 0. indexteki elemanını yani yönünü alacağım. (up-down-left-right)

    # Silah grafiği:
    # Silah png'lerimiz player'ın direction'ına göre ayarlandı. (up,down.. gibi)
    # Bu nedenle silah png'lerimize ulaşırken player direction'ını kullanacağız.
    # Hangi silahı seçeceğimizi belirlemek için player içerisinde değişkenlerimizi oluşturduk.(weapon ve weapon_index)
    full_path = f'graphics/weapons/{player.weapon}/{direction}.png' # weapons klasörü içerisinden index ile ulaştığımız silahın klasörünün içerisine giriyor ve yönümüze göre silah png'sini seçiyoruz.
    self.image = pygame.image.load(full_path).convert_alpha() # Silahımızın fotoğrafı için surface tutan bir değişken oluşturuyoruz ve yukarıda ulaştığımız silah yön ve konumuna uygun png'yi getiriyoruz.

    # Silah yerleştirimi:
    if direction == 'right': # Eğer yönümüz sağa doğruysa silahın rect'inin sol tarafı player rect'inin sağ tarafında olmalı.
      self.rect = self.image.get_rect(midleft=player.rect.midright + pygame.math.Vector2(0,16)) # Karakterin kolu resmin aşağısında olduğu için silahı biraz daha aşağıda oluşturuyoruz.
    elif direction == 'left': # Eğer yönümüz sola doğruysa silahın rect'inin sağ tarafı player rect'inin sol tarafında olmalı.
      self.rect = self.image.get_rect(midright=player.rect.midleft + pygame.math.Vector2(0,16)) # Karakterin kolu resmin aşağısında olduğu için silahı biraz daha aşağıda oluşturuyoruz.
    elif direction == 'up': # Eğer yönümüz yukarı doğruysa silahın rect'inin alt tarafı player rect'inin üst tarafında olmalı.
      self.rect = self.image.get_rect(midbottom=player.rect.midtop + pygame.math.Vector2(-10,0)) # Karakterin kolu biraz solda olduğu için biraz daha solda oluşturuyoruz.
    else: # Eğer yönümüz aşağı doğruysa silahın rect'inin üst tarafı player rect'inin alt tarafında olmalı.
      self.rect = self.image.get_rect(midtop=player.rect.midbottom + pygame.math.Vector2(-10,0)) # Karakterin kolu biraz solda olduğu için biraz daha solda oluşturuyoruz.

In [ ]:
# UI kısmını oluşturacağımız bölüm:

import pygame
from settings import *

class UI: # UI oluşturmak için bir UI sınıfı oluşturuyoruz.
  def __init__(self):
    
    # Genel bilgiler
    self.display_surface = pygame.display.get_surface() # Bir display ekranına ihtiyacımız var.
    self.font = pygame.font.Font(UI_FONT,UI_FONT_SIZE) # settings içerisindeki değişkenleri kullanarak fontu oluşturuyoruz.

    # bar ayarları
    self.health_bar_rect = pygame.Rect(10,10,HEALTH_BAR_WIDTH,BAR_HEIGHT) # can barı için bir rect oluşturuyoruz. (Rect(left,top,width,height))
    self.energy_bar_rect = pygame.Rect(10,34,ENERGY_BAR_WIDTH,BAR_HEIGHT) # enerji barı için bir rect oluşturuyoruz. (Rect(left,top,width,height))

    # Silah sözlüğünü listeye çevirme işlemi (Silah UI kısmı için.)
    self.weapon_graphics = [] # Silah grafiklerini tutacak olan listeyi oluşturuyoruz.
    for weapon in weapon_data.values(): # Silah bilgileri'nin value kısımlarıyla ilgileneceğim ve elime bir sözlük daha gelecek.
      path = weapon['graphic'] # Silah'ların grafik kısımlarını bir path'e atıyoruz.
      weapon = pygame.image.load(path).convert_alpha() # Silah fotoğraflarını bir surface haline getiriyorum ve weapon değişkenine atıyoruz.
      self.weapon_graphics.append(weapon) # Listeme bu silah surface'ımı ekliyorum.
    
    # Büyü sözlüğünü listeye çevirme işlemi (Büyü UI kısmı için.)
    self.magic_graphics = [] # Büyü grafiklerini tutacak olan listeyi oluşturuyoruz.
    for magic in magic_data.values(): # Büyü bilgileri'nin value kısımlarıyla ilgileneceğim ve elime bir sözlük daha gelecek.
      path = magic['graphic'] # Büyü'lerin grafik kısımlarını bir path'e atıyoruz.
      magic = pygame.image.load(path).convert_alpha() # Büyü fotoğraflarını bir surface haline getiriyorum ve magic değişkenine atıyoruz.
      self.magic_graphics.append(magic) # Listeme bu büyü surface'ımı ekliyorum.

  def show_bar(self,current,max_amount,bg_rect,color): # Bu esnek metot ile şuanki can ya da enerji, maks can ya da enerji, rect arkaplan rengi ve renk değişkenlerini alarak ekrana gösteren bir metot yazıyoruz.
    # arkaplanın çizilmesi:
    pygame.draw.rect(self.display_surface,UI_BG_COLOR,bg_rect) # pygame.draw.rect(surface,color,rect) parametrelerini alır.

    # Statların pixel'e dönüşümü:
    ratio = current / max_amount # ratio değişkeni pixellere göre canın durumunu çizmek için gerekli olan oran değişkenimiz.
    current_width = bg_rect.width * ratio # arka plandaki rect'in uzunluğu ile ratio oranını çarparak bir genişlik elde ediyorum. Bu da bizim statımızın şu anki değeri olacak.
    current_rect = bg_rect.copy() # arkadaki rect'i kopyaladım.
    current_rect.width = current_width # Ve kopyaladığım rect'in genişliğini aldığım değişkenle değiştirdim.

    # Barın çizilmesi:
    pygame.draw.rect(self.display_surface,color,current_rect) # pygame.draw.rect(surface,color,rect) parametrelerini alır.
    # color değişkeni parametre olan değişken.
    pygame.draw.rect(self.display_surface,UI_BORDER_COLOR,bg_rect,3) # Var olan rect üzerine (bg_rect yani arkaplan) yeni bir rect çiziyorum fakat satır genişliğini 3 vererek aslında bir çerçeve oluşturuyorum.
    # UI_BORDER_COLOR ise settings den geliyor.

  def show_exp(self,exp):
    text_surf = self.font.render(str(int(exp)),False,TEXT_COLOR) # self.font üzerinden render(info,AA(anti_aliasing(pixel olduğu için kullanmayacağız.)),color) metodunu çağırıyoruz.
    # TEXT_COLOR settings den gelen renk.
    # exp'yi ise player init içerisinde bulunan self.exp ile getireceğiz ve str'e çevireceğiz.
    x = self.display_surface.get_size()[0] - 20 # ekran botunu (x,y) ile geitirip x'ini aldık.
    y = self.display_surface.get_size()[1] - 20 # ekran botunu (x,y) ile geitirip y'ini aldık.
    text_rect = text_surf.get_rect(bottomright = (x,y))

    pygame.draw.rect(self.display_surface,UI_BG_COLOR,text_rect.inflate(20,20)) # text rect'imizin üzerine bir arkaplan rect'i çiziyoruz.
    # Fakat siyah arkaplanın biraz daha geniş olabilmesi için text_rect üzerine inflate metodunu uygulayabiliriz.

    pygame.draw.rect(self.display_surface,UI_BORDER_COLOR,text_rect.inflate(20,20),3) # text rect'imizin üzerine bir arkaplan rect'i çiziyoruz.
    # Arkaplanımız için bir çerçeve oluşturuyoruz. Son parametre bizim rect genişliğimiz.

    self.display_surface.blit(text_surf,text_rect) # Ve ekrana yazımız ve rect'ini çizdiriyoruz.

  def selection_box(self,left,top,has_switched): # Silah ve büyü seçenekleri için bir kutucuk oluşturacağız.
    # left ve top pozisyonları için gerekli olan parametreler.
    # has_switched parametresi ile de silah değiştiriyor muyum buna bakacağım. Eğer silah değiştiriyorsam silah UI çerçevesinin rengini silah değiştirme işlemi boyunca değiştireceğim.
    bg_rect = pygame.Rect(left,top,ITEM_BOX_SIZE,ITEM_BOX_SIZE) # pygame.Rect(left,top,w,h) parametrelerini alır. Arkaplan için bir rect oluşturuyoruz.
    # left ve top parametre olarak geliyor.
    # w, h değişkenleri için settingsden değişken getiriyoruz.
    pygame.draw.rect(self.display_surface,UI_BG_COLOR,bg_rect) # Oluşturduğumuz rect'i ekranımızıa verilen renkte çizdiriyoruz..

    if has_switched: # Eğer silah değiştirme işlemi yapıyorsak çerçeve rengini bu işlem boyunca farklı bir renk yapacağız.
      pygame.draw.rect(self.display_surface,UI_BORDER_COLOR_ACTIVE,bg_rect,3) # (Eğer silah değiştirmiyorsak) Oluşturduğumuz rect'in etrafına bir çerçeve oluşturuyoruz. Renk değiştirip en son da genişlik değerini veriyoruz.
    else:
      pygame.draw.rect(self.display_surface,UI_BORDER_COLOR,bg_rect,3) # (Eğer silah değiştirmiyorsak) Oluşturduğumuz rect'in etrafına bir çerçeve oluşturuyoruz. Renk değiştirip en son da genişlik değerini veriyoruz.
    
    return bg_rect # weapon_overlay metodu png'yi yerleştirmek için bg_rect'in pozisyonuna ihtiyaç duyuyor bu nedenle de bg_rect'i döndürmeli.

  def weapon_overlay(self,weapon_index,has_switched): # Bu metot ile hangi silah seçiliyse kutucuğa onun fotoğrafını koyacağız. Bu nedenle silah indeksini getirmemiz gerekiyor.
    # has_switched parametresi selection_box için gerekli olan parametre.
    bg_rect = self.selection_box(10,630,has_switched) # Silah kutucuğumuzu çizdiriyoruz.
    # selection_box metodu bg_rect'i döndürüyor ve biz de bu bg_rect'i kullanabiliriz. Değişkenin ismini değiştirmeden tekrardan atıyoruaz.
    weapon_surf = self.weapon_graphics[weapon_index] # Silah index'imize göre listeden doğru olan surface seçimini yapıp rect'e yerleştirecek.
    weapon_rect = weapon_surf.get_rect(center = bg_rect.center) # Ve silah rect'ini de bg_rect ile merkezleri çakışacak şekilde yerleştiriyorum.

    self.display_surface.blit(weapon_surf,weapon_rect) # display ekranımıza rect ve surface'ı çizdiriyoruz.
    # weapon_surf için settings içerisindeki weapon_data içindeki silah indeksine göre seçim yaptıktan sonra oradaki png'nin surface'a dönüşmüş halini getireceğim.
    # weapon_rect ise merkezi bg_rect ile aynı yerde olacak olan rect olacak.

  def magic_overlay(self,magic_index,has_switched): # Bu metot ile hangi büyü seçiliyse kutucuğa onun fotoğrafını koyacağız. Bu nedenle büyü indeksini getirmemiz gerekiyor.
    # has_switched parametresi selection_box için gerekli olan parametre.
    bg_rect = self.selection_box(80,635,has_switched) # Büyü kutucuğumuzu çizdiriyoruz.
    # selection_box metodu bg_rect'i döndürüyor ve biz de bu bg_rect'i kullanabiliriz. Değişkenin ismini değiştirmeden tekrardan atıyoruaz.
    magic_surf = self.magic_graphics[magic_index] # Büyü index'imize göre listeden doğru olan surface seçimini yapıp rect'e yerleştirecek.
    magic_rect = magic_surf.get_rect(center = bg_rect.center) # Ve Büyü rect'ini de bg_rect ile merkezleri çakışacak şekilde yerleştiriyorum.

    self.display_surface.blit(magic_surf,magic_rect) # display ekranımıza rect ve surface'ı çizdiriyoruz.
    # magic_surf için settings içerisindeki magic_data içindeki büyü indeksine göre seçim yaptıktan sonra oradaki png'nin surface'a dönüşmüş halini getireceğim.
    # magic_rect ise merkezi bg_rect ile aynı yerde olacak olan rect olacak.

  def display(self,player): # oluşturduğumuz rect'leri ekrana çizmemizi sağlayan metot.
    # fakat can barını burada çizdirmeyeceğiz.
     # pygame.draw.rect(self.display_surface,'black',self.health_bar_rect) # pygame.draw.rect(surface,color,rect) parametrelerini alır.
    self.show_bar(player.health,player.stats['health'],self.health_bar_rect,HEALTH_COLOR)
    # player.health ile player'ın şuanki can durumunu alıyoruz.
    # player.stats['health'] ile player maks can değerini alıyoruz.
    # self.health_bar_rect ile can barının bulunacağı yeri alıyoruz.
    # HEALTH_COLOR ile can rengini alıyoruz.
    self.show_bar(player.energy,player.stats['energy'],self.energy_bar_rect,ENERGY_COLOR)
    # player.energy ile player'ın şuanki enerji durumunu alıyoruz.
    # player.stats['energy'] ile player maks enerji değerini alıyoruz.
    # self.energy_bar_rect ile can barının bulunacağı yeri alıyoruz.
    # ENERGY_COLOR ile can rengini alıyoruz.

    self.show_exp(player.exp) # Show_exp metodu ile de tecrübe puanımızı yazdıran bir metot yazacağız.
    # player'ı display metodu parametresinden getirdik.

    self.weapon_overlay(player.weapon_index,not player.can_switch_weapon) # Silahımızı ve büyümüzü ekrana çizdiriyoruz.
    # weapon_index ve can_switch_weapon parametrelerini de veriyoruz. (can_switch_weapon'ı not verip alıyoruz çünkü silah değiştirebilme durumunda değil silah değiştirirken yani değiştirememe durumunda renk değişimi yapacağız.)
    self.magic_overlay(player.magic_index,not player.can_switch_magic) # Aynısından bir de büyü kutucuğu için çizdiriyoruz.

In [ ]:
# Entity dosyası

# Bu dosya içerisinde düşmanlar ve karakter için aynı olan move ve collision metotlarını entity class'ı içerisine alıp ikisi için de kullanacağız.

import pygame
from math import sin

class Entity(pygame.sprite.Sprite): # Entity class'ımızı oluşturuyoruz ve kalıtımını Sprite'lardan almasını istiyoruz.
  def __init__(self,groups): # init metodumuzu yazıyoruz.
    super().__init__(groups) # sprite sınıfını initialize ediyoruz.
    self.frame_index = 0
    self.animation_speed = 0.15
    self.direction = pygame.math.Vector2()
  
  def move(self,speed): # Entity'nin yürümesini sağlayacak metodu oluşturuyoruz.
    # speed parametresi entity'nin hızı olacak.

    # Fakat (1,1) gibi çift yönlü durumlarda entity aşırı hızlı gidiyor bu sebeple bu hızı normalleştirmemiz gerek:

    if self.direction.magnitude() != 0: # Direction'ımızın bir değeri olup olmadığına bakıyoruz.(x ve y vektörleri için)
      self.direction = self.direction.normalize() # Eğer öyleyse bu hızı normalleştiriyoruz.(0.7 gibi bir değer)

     # self.rect.center += self.direction * speed # entity rectangle'ının merkezini her saniye entitynin directionını, entity hızı ile çarpıp ekliyoruz.
    # Bu şekilde eğer yönümüz sola (-1,0) ise hız 5 ise (-5,0) kordinatlarına taşıyor ve hızlıca entityi çiziyoruz. Böylece entity hareket ediyor.

    # 1- # Yönleri ayırdık çünkü hangi yönder çarpışma olduğunu bilmek istiyoruz.
    """
      self.rect.x += self.direction.x * speed # entity rect'inin x konumuna direction * speed ekliyoruz.
      self.collision('horizontal') # Yatay çarpışmanın kontrolünü yapıyoruz.
      self.rect.y += self.direction.y * speed # entity rect'inin y konumuna direction * speed ekliyoruz.
      self.collision('vertical') # Dikey çarpışmanın kontrolünü yapıyoruz.
    """    
    # 2- # rect'leri hitboxlar ile değiştiriyoruz.

    self.hitbox.x += self.direction.x * speed # entity hitbox'inin x konumuna direction * speed ekliyoruz.
    self.collision('horizontal') # Yatay çarpışmanın kontrolünü yapıyoruz.
    self.hitbox.y += self.direction.y * speed # entity hitbox'inin y konumuna direction * speed ekliyoruz.
    self.collision('vertical') # Dikey çarpışmanın kontrolünü yapıyoruz.
    # Yönleri ayırdık çünkü hangi yönder çarpışma olduğunu bilmek istiyoruz.
    self.rect.center = self.hitbox.center # Rect'in merkeziyle hitbox'ın merkezini aynı yere koyduk ve ayarladık.

  def collision(self,direction): # Çarpışmaları gerçekleştirecek metodu yazıyoruz.
    # direction parametresi ile çarpışmanın hangi yönden olduğuna bakıyoruz.

  # 1- 
  """
    if direction == 'horizontal': # Eğer yatay eksendeysek.
      for sprite in self.obstacle_sprites: # obstacle_sprites grubundaki her bir sprite için
        if sprite.rect.colliderect(self.rect): # Eğer sprite karakterimiz ile çarpışıyorsa.
          if self.direction.x > 0: # Eğer entity yönü sağa doğruysa
            self.rect.right = sprite.rect.left # Çarpışmadan sonra entity rect'inin sağ tarafı, çarptığımız nesnenin rect'inin sol tarafının konumunda olmalı.
          if self.direction.x < 0: # Eğer karakterimiz sola doğruysa
            self.rect.left = sprite.rect.right # Çarpışmadan sonra entity rect'inin sol tarafı, çarptığımız nesnenin rect'inin sağ tarafının konumunda olmalı.
    
    if direction == 'vertical': # Eğer dikey eksendeysek.
      for sprite in self.obstacle_sprites: # obstacle_sprites grubundaki her bir sprite için
        if sprite.rect.colliderect(self.rect): # Eğer sprite entity ile çarpışıyorsa.
          if self.direction.y > 0: # Eğer entity yönü aşağı doğruysa
            self.rect.bottom = sprite.rect.top # Çarpışmadan sonra entity rect'inin alt tarafı, çarptığımız nesnenin rect'inin üst tarafının konumunda olmalı.
          if self.direction.y < 0: # Eğer entity yukarı doğruysa
            self.rect.top = sprite.rect.bottom # Çarpışmadan sonra entity rect'inin üst tarafı, çarptığımız nesnenin rect'inin alt tarafının konumunda olmalı.
  """
  # 2- rect'leri hitbox ile değiştirdik ki overlap yapabilelim.
    
    if direction == 'horizontal': # Eğer yatay eksendeysek.
      for sprite in self.obstacle_sprites: # obstacle_sprites grubundaki her bir sprite için
        if sprite.hitbox.colliderect(self.hitbox): # Eğer sprite entity hitboxı ile çarpışıyorsa.
          if self.direction.x > 0: # Eğer entity yönü sağa doğruysa
            self.hitbox.right = sprite.hitbox.left # Çarpışmadan sonra entity hitbox'ının sağ tarafı, çarptığımız nesnenin hitbox'inin sol tarafının konumunda olmalı.
          if self.direction.x < 0: # Eğer karakterimiz sola doğruysa
            self.hitbox.left = sprite.hitbox.right # Çarpışmadan sonra entity hitbox'inin sol tarafı, çarptığımız nesnenin hitbox'inin sağ tarafının konumunda olmalı.
    
    if direction == 'vertical': # Eğer dikey eksendeysek.
      for sprite in self.obstacle_sprites: # obstacle_sprites grubundaki her bir sprite için
        if sprite.hitbox.colliderect(self.hitbox): # Eğer sprite karakterimiz ile çarpışıyorsa.
          if self.direction.y > 0: # Eğer entity yönü aşağı doğruysa
            self.hitbox.bottom = sprite.hitbox.top # Çarpışmadan sonra entity hitbox'inin alt tarafı, çarptığımız nesnenin hitbox'inin üst tarafının konumunda olmalı.
          if self.direction.y < 0: # Eğer karakterimiz yukarı doğruysa
            self.hitbox.top = sprite.hitbox.bottom # Çarpışmadan sonra entity hitbox'inin üst tarafı, çarptığımız nesnenin hitbox'inin alt tarafının konumunda olmalı.

  def wave_value(self): # Entity'lerin hasar alınca transparan duruma geçmesi için gerekli olan alpha değerini 0-255 değeri arasında değiştiren metot.
    # alpha değeri için sinus dalgasını kullanacağız. Sinüs dalgası pozxitif değer alırken 255, negatif değer alırken 0 değerini alacak.
    value = sin(pygame.time.get_ticks()) # Zaman geçtikçe bir sinüs değeri alıyoruz.
    if value >= 0: # Eğer bu değer 0'dan büyükse
      return 255 # 255 döndür
    else: # Eğer 0'dan küçükse
      return 0 # 0 döndür.

In [ ]:
# Düşmanlar

import pygame
from settings import *
from entity import Entity
from support import *

class Enemy(Entity): # Bir düşman sınıfı oluşturuyoruz ve Entity'den kalıtım alıyor.
  def __init__(self,monster_name,pos,groups,obstacle_sprites,damage_player,trigger_death_particles,add_exp):
    # monster_name canavarın ismi olacak ve enemy_data içerisindeki key'lerden gelecek.
    # pos düşmanın oyun başında nerede konumlanacağını belirtecek.
    # groups ise hangi sprite grubunda olacağını belirtecek.
    # Ve tekrar collision için tüm obstacle_sprites'lara bakıyoruz ve bunu düşmanlar için de tanımlamalıyız.
    # damage_player ise damage_player'ı tutacak olan değişken. Bu metod ile player'a hasar verilebilecek.
    # trigger_death_particles ise öldüğünde animasyonları oluşturacak metot.
    # add_exp ile öldüklerinde exp vermelerini sağlıyoruz.

    # Genel Ayarlar
    super().__init__(groups) # Kalıtım aldığı sınıfı init ediyoruz.
    self.sprite_type = 'enemy' # Eğer bir düşmana saldırıyorsak canı azalacak ya da ölecek. Fakat bir Tile'a saldırıyorsak o bizle etkileşime girmeyecek sadece yok olacak.

    # grafik ayarları
    self.import_graphics(monster_name) # monster_name parametresinden gelen canavar ismine göre grafik yüklemesi yapan metotu çağırıyorum.
    self.status = 'idle' # Canavarın default durumu idle olsun istiyoruz.
    # self.image = pygame.Surface((64,64)) # Düşmanımız için geçici olarak bir siyah kare oluşturduk. (Eski)
    self.image = self.animations[self.status][self.frame_index] # animasyonlar dict içerisinden, canavarın durumuna göre png getiriyoruz ve frame_index'imize göre Surface'lardan birini seçiyoruz.
    
    # Canavar hareketleri
    self.rect = self.image.get_rect(topleft = pos) # Ve düşman png'mizin bir rect'ini oluşturduk.
    self.hitbox = self.rect.inflate(0,-10) # Canavarlar için bir hitbox oluşturduk. Entity class'ı hareket ve collision için hitbox'a ihtiyaç duyuyor.
    self.obstacle_sprites = obstacle_sprites # Ve tekrar collision için tüm obstacle_sprites'lara bakıyoruz ve bunu düşmanlar için de oluşturmalıyız.

    # Canavar bilgileri
    self.monster_name = monster_name # Canavar ismini alan değişken
    monster_info = monster_data[self.monster_name] # Canavar bilgilerini getiren değişken
    self.health = monster_info['health'] # Yukarıdaki değişken üzerinden can bilgisini aldık.
    self.exp = monster_info['exp'] # Canavar exp'si
    self.speed = monster_info['speed'] # Canavar hızı
    self.attack_damage = monster_info['damage'] # Canavar saldırı hasarı
    self.resistance = monster_info['resistance'] # Canavar geri sekmesi
    self.attack_radius = monster_info['attack_radius'] # Canavar saldırı alanı
    self.notice_radius = monster_info['notice_radius'] # Canavar fark etme alanı
    self.attack_type = monster_info['attack_type'] # Canavar saldırı türü.

    # Player ile etkileşim
    self.can_attack = True # Sürekli player'a vurulu halde kalmasın diye bir değişken oluşturuyoruz.
    self.attack_time = None # Düşmanın tekrar saldırabilmesi için bir timer oluşturacağız. Bu da saldırdığı andan itibaren hesaplanan süre olacak.
    self.attack_cooldown = 400 # Tekrar saldırı yapılabilmesi için gereken süre.
    self.damage_player = damage_player
    self.trigger_death_particles = trigger_death_particles
    self.add_exp = add_exp

    # Ölümsüzlük zamanlayıcısı
    self.vulnerable = True # Hasar alabilir mi?
    self.hit_time = None # Timer için hesaplanacak vuruş süresi
    self.invincibility_duration = 300 # Her çarpışmadan sonraki hasar almama süresi.

    # Sesler
    self.death_sound = pygame.mixer.Sound("audio/death.wav") # Canavar ölüm sesi
    self.hit_sound = pygame.mixer.Sound("audio/hit.wav") # Canavar hasar yeme sesi
    self.attack_sound = pygame.mixer.Sound(monster_info['attack_sound'])
    # monster_info, monster_data içerisinden seçili canavarın değerlerini getirir.
    # ardından da bu dict içerisinden saldırı sesini getireceğiz.

    self.death_sound.set_volume(0.2) # Ses düzeyi ayarı
    self.hit_sound.set_volume(0.2) # ses düzeyi ayarı
    self.attack_sound.set_volume(0.3)

  def import_graphics(self,name): # Canavar animasyonlarını getiren metod.
    self.animations = {'idle': [], 'move': [], 'attack': []} # animasyonları tutacak olan dict.
    # Eğer canavar bizim etki alanımız dışında ise idle olacak.
    # Yakınındaysak bize doğru hareket edecek.
    # Çok yakındaysak saldıracak.
    # Her bir canavar 3 farklı dosya içeriyor isimleri idle move ve attack. Her bir dosyada da farklı o işlem animasyonları var.

    main_path = f'graphics/monsters/{name}/' # Bu path ile hangi canavar ismi verildiyse onun path'ine gideceğiz. # Monster_name'ini bir Enemy oluştururken vereceğiz.
    for animation in self.animations.keys(): # Yukarıdaki listedeki durumlar için
      self.animations[animation] =  import_folder(main_path + animation) # Her bir durum için örneğin idle durumu için listeyi dolduruyoruz.
      # import_folder metodunu tekrar kullanabiliriz. Örneğin graphics/monsters/raccoon/idle klasörüne gidip tüm png'leri bir surface yapacak. Aynısını move ve attack için de yapacak.

  def get_player_distance_direction(self,player): # Player mesafesini ve yönünü alan metot.
    enemy_vec = pygame.math.Vector2(self.rect.center) # Canavarın merkezini bir vektöre dönüştürüyoruz.
    player_vec = pygame.math.Vector2(player.rect.center) # Player'ın merkezini bir vektöre dönüştürüyoruz.
    distance = (player_vec - enemy_vec).magnitude() # Vektör farkını alıyoruz fakat bu bize bir vektör veriyor.
    # Mesafeyi bulabilmek için ise magnitude metodunu kullanıyoruz.

    if distance > 0: # normalize metodu 0 ile çalışamayacağı için sıfır dışındaki durumlar için kullanıyoruz.
      direction = (player_vect - enemy_vect).normalize() # Aynı fark vektörünü biz hızla çarpıp hareket sağlayacağız.
    # Fakat bu vektör hızla çarpılınca çok büyük bir değer geliyor. Bu sebeple normalize metodunu kullanacağız.
    else: # Eğer distance = 0 ise player ve canavar üst üste demektir yani canavar durabilir.
      direction = pygame.math.Vector2()

    return (distance,direction)

  def get_status(self,player): # Bu metot ile player'ın konumunu alabilecek ve player konumuna göre canavarın ne yapması gerektiğini alacağız.
    distance = self.get_player_distance_direction(player)[0] # Bu değişken player ile canavar arası mesafeyi ölçecek.

    if distance <= self.attack_radius and self.can_attack: # Eğer mesafe, saldırı mesafesinden az ya da eşitse ve canavar saldırabilecek durumda ise,
      if self.status != 'attack': # attack durumu sona erdiğinde.
        self.frame_index = 0 # animasyon bittikten sonra tekrar 0 yapıyorum ki sürekli son png'de kalmasın.
      self.status = 'attack' # Canavar durumunu saldır yap.
    elif distance <= self.notice_radius: # Eğer mesafe, fark etme mesafesinden az ya da eşitse,
      self.status = 'move' # Canavar durumunu hareket et yap.
    else: # Eğer mesafe çok uzaksa,
      self.status = 'idle' # Canavar durumunu boş dur yap.

  def actions(self,player):
    if self.status == 'attack': # Eğer saldırı durumundaysa saldır.
      # print('attack') # Eğer ben buraya direkt self.can_attack = False yaparsam saldırı animasyonu bitmeden saldırı sona erer. Bu nedenle saldırı animasyonu bittikten sonra bunu yapmalıyım.
      self.attack_time = pygame.time.get_ticks() # Saldırı durumu başladığından itibaren zamanı tutmaya başlayacak.
      self.damage_player(self.attack_damage,self.attack_type) # Ve player'a hasar vereceğiz. Amount ve attack_type değişkenleri lazım
      # Parametre olarak yukarıda oluşturduğum değişkenleri veriyoruz.  
      self.attack_sound.play() # Ve saldırı ses efektini çalıyoruz. 
    elif self.status == 'move': # Eğer hareket et durumundaysa hareket et.
      self.direction = self.get_player_distance_direction(player)[1] # canavarın yönü için metodun getirdiği 1. indexi alıyoruz.
    else: # Eğer hiçbiri değilse en sonki konumunda kal.
      self.direction = pygame.math.Vector2() # Direction 0 oluyor ve hareket etmiyor.

  def animate(self):
    animation = self.animations[self.status] # Canavarın şu anki durumu

    self.frame_index += self.animation_speed # Entity'den gelen değişkenlerle animasyon hızımızı ayarlıyoruz.
    if self.frame_index >= len(animation): # Eğer fame_index'imiz örneğin idle için png sayısından fazla ise 0 yapacağız.
      if self.status == 'attack': # Ve eğer saldırıyorsak. Yani 4 saldırı fotomuz olsun. Bizim index'imiz 4 ün üstüne çıktığı an saldırı bitmiş demektir. Bu nedenle burada can_attack = False yapabiliriz.
        self.can_attack = False
      self.frame_index = 0
    
    self.image = animation[int(self.frame_index)] # Canavar png'sini sıra ile alıyorum.
    self.rect = self.image.get_rect(center = self.hitbox.center) # Ve rect'i yerleştiriyoruz.

    # Düşmanın hasar alınca titremesi:
    if not self.vulnerable: # Eğer düşman hasar almışsa:
      alpha = self.wave_value() # 0 ile 255 arasında değişen bir transparanlık değeri lazım ve bunu da Entity sınıfı içerisinde oluşturacağız. Çünkü hem player hem de düşmanlar hasar aldığında transparan hale geçmeli.
      self.image.set_alpha(alpha) # Ve surface a uyguluyoruz.
    else: # Eğer düşman hasar almamışsa normal durumda.
      self.image.set_alpha(255) # Transparanlık ekliyoruz. 255 maksimum değer yani tamamen görünür halde.

  def cooldowns(self): # Bu metot ile düşmanın tekrar saldırabilmesi için gereken süreyi ayarlayacak ve bir timer oluşturacağız.
    current_time = pygame.time.get_ticks() # Oyunda sürekli hesaplanan geçerli zaman
    
    # Düşman tekrar saldırma Timer'ı
    if not self.can_attack: # Eğer self.attack = False ise
      if current_time - self.attack_time >= self.attack_cooldown: # Eğer cooldown sağlanmışsa
        self.can_attack = True # Tekrar saldırabilir hale getir.
      # self.attack_time'ı ise actions içerisinden alacağız.
    
    # Düşmanın tekrardan hasar alabilme Timer'ı:
    if not self.vulnerable: # Eğer False ise hasar alma durumu
      if current_time - self.hit_time >= self.invincibility_duration: # Ve eğer cooldown sağlanmış ise
        self.vulnerable = True # Hasar alabilmeyi tekrar True yap.

  def get_damage(self,player,attack_type): # Canavarın hasar almasını sağlayan metot. Fakat pygame collision'ı saniyede 60 kere algıladığı için direkt ölüyorlar. Bu sebeple bir Timer oluşturmamız lazım.
    # player parametresi self.player'ı alacak.
    # attack_type ise attack_sprite.sprite_type'ı tutacak ki büyü mü yoksa silah mı kullandık bilelim.
    if self.vulnerable: # Eğer canavar hasar alabiliyor ise.
      self.hit_sound.play() # Hasar yeme ses efektini çal.
      self.direction = self.get_player_distance_direction(player)[1] # Player directionunu aldık.
      if attack_type == 'weapon': # Eğer silah ile vuruş yapmışsak.
        self.health -= player.get_full_weapon_damage() # Düşman canını get_full_weapon_damage metodu ile getirdiğim toplam silah hasarı kadar azaltacağım. Player içerisindeki
        # Bu metodu player içerisinde oluşturacağım.
      else: # Büyü hasarı ise
        self.health -= player.get_full_magic_damage() # Düşman canını get_full_magic_damage metodu ile getirdiğim toplam büyü hasarı kadar azaltacağım. Player içerisindeki
      
      self.hit_time = pygame.time.get_ticks() # Ve atak yaptıktan sonra da zaman tutmaya başlıyoruz.
      self.vulnerable = False # Her işlemden sonra tekrar hasar almayı False yapıyorum ve timer ile bunu bir süre sonra (self.invinciblitiy_time kadar) tekrar True yapıcam.

  def check_death(self): # Eğer canavarın canı 0'ın altına düşmüşse ölümünü kontrol eden metot.
    if self.health <= 0: # Eğer canı 0 veya altındaysa,
      self.kill() # Yok et.
      self.trigger_death_particles(self.rect.center,self.monster_name) # Level class'ı içerisindeki trigger_death_particles metodunu çağırıyoruz ki düşman öldüğünde efekt çıkarsın. (pos, particle_type) parametrelerini alıyor.
      # Particle düşmanın merkezinde oluşacak ve particle types için ise canavar ismini kullanacağız. Çünkü particles içerisindeki dict e baktığımızda canavar ölüm efektleri isimleriyle adlandırılmış.
      self.add_exp(self.exp) # Ve Level içerisinde oluşturduğum öldüklerinde exp vermelerini sağlayan metodu çağırıyorum.
      # self.exp ile her bir canavarın exp'sini veriyoruz.
      self.death_sound.play() # Ölme ses efektini çalıyoruz.

  def hit_reaction(self): # Bu metot ile düşman hasar aldığında geri savrulmasını sağlayacağız.
    if not self.vulnerable: # Eğer düşman daha yeni hasar almış ise.
      self.direction *= -self.resistance # Düşmanın yönünü resistance değerinin negatifi ile çarparak arka tarafa doğru yönlenmesini sağlıyorum.
    # tekrar hasar alabilir duruma gelene kadar direction ters yönde olacak ve savrulacak.

  def update(self):
    self.hit_reaction() # Hasar aldığında geriye savrulsun.
    self.move(self.speed) # Move metodunu entity class'ı üzerinden alıyoruz ve değişken olarak canavar hızını veriyoruz.
    self.animate() # Animasyonları çağırıyoruz.
    self.cooldowns() # Saldırı cooldown'ını çağırıyoruz.
    self.check_death() # Canavarın ölüp ölmediğini kontrol ediyoruz.

  def enemy_update(self,player): # Düşman durumlarını kontrol eden metot. Level class'ı içerisinde çağırdık ve bu şekilde player'a ulaşabildik.
    self.get_status(player)
    self.actions(player)


In [ ]:
# Oyuncu ve düşman etkileşimi

## Oyuncu saldırı yaparken
# Öncelikle 2 tane daha yeni Sprite grubu oluşturacağız. (attack sprites and attackable sprites)
# Attack sprites'ı içerisine silahlarımızı ve büyülerimizi, attackable sprites içerisine ise düşmanları ve objeleri koyacağız.
# Eğer bu iki sprite grubu arasında herhangi bir collide var ise sprite_type'ına bakarak sonrasında ne olacağını belirtceğiz.
# Eğer sprite_type == 'enemy' ise düşmanın canı azalacak. Eğer 'grass' ise grass sprite'ı yok olacak.

## Düşman saldırı yaparken
# Düşman saldırı yaparken sadece attack durumuna bakacağız ve gerçekleşmişse player'ın canını azaltacağız.

In [ ]:
# Partiküller

# Partiküller da aslında sadece animasyonu olan sprite'lardır.
# Tek farkı animasyon sona erdiğinde yok olmaları gerekir.

# partikül dosyaları oyun başladığı gibi import edilmeli yoksa oyunu yavaşlatır.


In [ ]:
# Patiküller

import pygame
from support import import_folder
from random import choice

class AnimationPlayer:
  def __init__(self):
    self.frames = {
			# Büyü animasyonları
			'flame': import_folder('graphics/particles/flame/frames'), # flame büyüsü efektleri
			'aura': import_folder('graphics/particles/aura'), # aura büyüsü efektleri
			'heal': import_folder('graphics/particles/heal/frames'), # heal büyüsü efektleri
			
			# Saldırı animasyonları 
			'claw': import_folder('graphics/particles/claw'), # Claw atağı animasyonları
			'slash': import_folder('graphics/particles/slash'), # slash atağı animasyonları
			'sparkle': import_folder('graphics/particles/sparkle'), # sparkle atağı animasyonları
			'leaf_attack': import_folder('graphics/particles/leaf_attack'), # leaf_attack atağı animasyonları
			'thunder': import_folder('graphics/particles/thunder'), # thunder atağı animasyonları

			# Canavar ölüm animasyonları
			'squid': import_folder('graphics/particles/smoke_orange'),
			'raccoon': import_folder('graphics/particles/raccoon'),
			'spirit': import_folder('graphics/particles/nova'),
			'bamboo': import_folder('graphics/particles/bamboo'),
			
			# Yapraklar yokolma animasyonları 
			'leaf': (
				import_folder('graphics/particles/leaf1'),
				import_folder('graphics/particles/leaf2'),
				import_folder('graphics/particles/leaf3'),
				import_folder('graphics/particles/leaf4'),
				import_folder('graphics/particles/leaf5'),
				import_folder('graphics/particles/leaf6'),
				self.reflect_images(import_folder('graphics/particles/leaf1')),
				self.reflect_images(import_folder('graphics/particles/leaf2')),
				self.reflect_images(import_folder('graphics/particles/leaf3')),
				self.reflect_images(import_folder('graphics/particles/leaf4')),
				self.reflect_images(import_folder('graphics/particles/leaf5')),
				self.reflect_images(import_folder('graphics/particles/leaf6'))
				)
			}
    # pygame bir şeyi yok ettiğikten sonra yeniden import ederken oyunu oldukça yavaşlatır. Bu nedenle yeni bir class oluşturduk.

  def reflect_images(self,frames):
    new_frames = [] # Dönmüş frame'leri tutan bir liste oluşturduk.
    for frame in frames: # Frames içerisindeki her bir frame için.
    # Bu frames ise yukarıdaki dict içerisinde tek tek yüklediğimiz animasyonlar için verildi.
      flipped_frame = pygame.transform.flip(frame,True,False) # Flip metodu bir nesneyi döndürmeye yarıyor.
      # True x ekseninde döndürmek için, False ise y ekseninde döndürmemek için verildi.
      new_frames.append(flipped_frame) # Dönmüş frame'leri listeye ekliyoruz.
    return new_frames # Ve bu listeyi döndürüyoruz.

  def create_grass_particles(self,pos,groups):
    animation_frames = choice(self.frames['leaf']) # animasyon frame'i olarak yukarıdaki dict içerisinden, leaf tuple'ından rastgele bir liste seçiyoruz.

    ParticleEffect(pos,animation_frames,groups) # Bir partikül efekti oluşturuyoruz.
    # Partikül efekti için position'ı Level içerisinde grass yok olurken verdik.(hedef sprite merkez konumu)
    # animation frames için de yukarıda rastgele bir seçim yaptık ve bir liste aldık.
    # groups ise Level içerisinde visible_sprites olarak verdik ki görebilelim.
    # Paritcle effect ise bunları kullanıp animasyonu gerçekleştirecek.

  def create_particles(self,animation_type,pos,groups): # Karakterin ne tür bir hasar aldığına bağlı olarak oluşacak animasyon metodu.
    animation_frames = self.frames[animation_type] # Yukarıdaki dict içerisinden animasyon türüne göre bir liste getireceğiz. Framelerden oluşan
    ParticleEffect(pos,animation_frames,groups) # Ve particleEffect classından çağırıyorum.
  # Enemies içerisinde actions kısmında her saldırı yapıldığında damage_player çağırılıyor. Ve damage player 2 parametre alıyor. Biri hasar diğeri de hasar türü. Biz hasar türünü kullanacağız.

class ParticleEffect(pygame.sprite.Sprite): # Partiküllerin bulunacağı sınıf
  def __init__(self,pos,animation_frames,groups):
    # pos partikülün konumunu belirtecek.
    # animation_frames ise her bir partikülün oluşması için gereken frame'ler
    # groups ise hangi grupta bulunacağı.
    super().__init__(groups) # Sprite class'ını init ediyoruz.
    self.sprite_type = 'magic' # Düşman hasar aldığında Enemy içerisindeki get_damage metodu çalışıyor fakat büyülerimizin sprite_type'ı yok. Bu nedenle burada bir sprite_type oluşturduk.
    self.frame_index = 0 # Her bir frame'i dönecek olan index'imiz.
    self.animation_speed = 0.15 # Her bir frame'i dönme hızımız.
    self.frames = animation_frames # Animasyon frame'lerini tutan değişken
    self.image = self.frames[self.frame_index] # Ve animasyon frame'i için aldığımız frame içerisinden frame indeximiz ile sırayla seçiyoruz.
    self.rect = self.image.get_rect(center = pos) # Ve frame'lerin rect'ini oluşturuyoruz. Ve rect'in konumu her birinin merkezi olacak.

  def animate(self): # Animasyonları gerçekleştirecek metot.
    self.frame_index += self.animation_speed # Animasyon geçiş hızını frame index'imize ekliyoruz.
    if self.frame_index >= len(self.frames): # Eğer indeksimiz listemizin uzunluğunun üstüne çıkarsa 
      self.kill() # O sprite'ı kaldıracağız.
    else: # Eğer çıkmaz ise
      self.image = self.frames[int(self.frame_index)] # Fotoğraf olarak frames'ler içerisinden index ile seçeceğiz.
    
  def update(self): # partiküllerin durumunu güncelleyen metot.
    self.animate() # Animasyonları çağırıyoruz.

In [ ]:
# Büyüler

# Flame büyüsü hasar verdiği için yine attack sprites içerisinde olacak.

# Heal büyüsü ise sadece birkaç particle çıkarıp player canını bir miktar yükseltecek.

# Flame büyüsü için player'ın yönüne göre 5 tane yan yana ateş sprite'ı oluşacak ve bu ateş spritelarına
# her birine ayrı rastgele bir offset vereceğiz ki daha doğal dursun.

import pygame
from settings import *
from random import randint

class MagicPlayer():
  def __init__(self,animation_player): # particle efektlerini tutacak animation_player'ı aldık.
    self.animation_player = animation_player

    # Sesler
    self.sounds = {
        'heal': pygame.mixer.Sound('audio/heal.wav'), # Heal sesi
        'flame': pygame.mixer.Sound('audio/Fire.wav') # Flame sesi
    }

  def heal(self,player,strength,cost,groups): # Heal büyüsü için bir fonksiyon yapıyoruz.
    # player ile player'ın canını artıracağımız için ihtiyaç duyuyoruz.
    # strength
    # cost ile büyü maliyetine bakıyoruz.
    # groups ile hangi gruplarda olduğuna bakacağız.
    if player.energy >= cost: # Eğer karakterin enerjisi yetiyorsa çağırabiliriz büyüyü.
      self.sounds['heal'].play() # heal sesini çağır.

      player.health += strength # karakterin canını büyünün gücü kadar arttırıyoruz.
      player.energy -= cost # Karakterin enerjisini büyü maliyeti kadar azaltıyoruz.
      if player.health >= player.stats['health']: # Eğer karakterin canı 100 üstüne çıkarsa
        player.health = player.stats['health'] # Player canını maks can yap

    # particles
      self.animation_player.create_particles('aura',player.rect.center,groups)
      # create_particles 3 parametre alıyor. aura animasyon türü ve dict içerisinden alıyoruz.(Particles içerisinden). ikinci parametre ile konumunu veriyoruz ve hangi grupta olacağını veriyoruz.
      self.animation_player.create_particles('heal',player.rect.center,groups)
      # Ve iyileşme animasyonumuzu da alıyoruz.

  def flame(self,player,cost,groups): # Flame büyüsü için bir fonksiyon yazıyoruz.
    # Bu metodu Level içerisinde create_magic içerisinde çağıracağız.
    if player.energy >= cost: # Eğer enerjimiz yeterli ise,
      self.sounds['flame'].play() # flame sesini çal.
      player.energy -= cost # Enerjiyi bu kadar azalt

      if player.status.split("_")[0] == "right": # Eğer sağa dönük isek
        direction = pygame.math.Vector2(1,0) # Sağ vektör
      elif player.status.split("_")[0] == "left": # Eğer sola dönük isek
        direction = pygame.math.Vector2(-1,0) # Sol vektör
      elif player.status.split("_")[0] == "up": # Eğer yukarı dönük isek
        direction = pygame.math.Vector2(0,-1) # Yukarı vektör
      else: # Eğer aşağı dönük isek
        direction = pygame.math.Vector2(0,1) # Aşağı vektör

      # Yan yana 5 ateş oluşturmak için aşağıdaki i değerini kullanacağız.
      # Örneğin sana doğru oluşturmak istiyoruz. Ve direction'ımız (1,0)
      # Önce bu direction'ı i ile çarpacağız ve ardından da TILESIZE ile çarpacağız ve yan yana 5 ateş oluşacak.

      for i in range(1,6): # 5 adet ateş oluşturacağız. Bu i değişkeni ile offset vereceğimiz için 1'den başlattık.
        if direction.x: # Horizontal (yatay eksende) Yani direction'ın x değeri 0'dan farklı ise
          offset_x = (direction.x * i) * TILESIZE  # Yan yana oluşturacak olan offset
          x = player.rect.centerx + offset_x + randint(-TILESIZE // 3, TILESIZE // 3) # Her bir flame'in x pozisyonu. Player'ın merkezinin x'i + offset'in x'i kadar olmalı. Ayrıca random bir offset daha veriyoruz ki tahmin edilebilir olmasın.
          y = player.rect.centery + randint(-TILESIZE // 3, TILESIZE // 3) # Her bir flame'in y pozisyonu. Ayrıca random bir offset daha veriyoruz ki tahmin edilebilir olmasın.
          self.animation_player.create_particles('flame',(x,y),groups) # Ve animasyonunu oluşturuyoruz.
          # dict içerisinden 'flame' i alacağız.
          # Konum olarak x ve y'yi verdik.
          # Grup olarak da parametreden gelen değeri veriyoruz.

        else: # Vertical (dikey eksende ise) yani x değeri 0 ise.
          offset_y = (direction.y * i) * TILESIZE  # Yan yana oluşturacak olan offset
          x = player.rect.centerx + randint(-TILESIZE // 3, TILESIZE // 3) # Her bir flame'in x pozisyonu. Player'ın merkezinin x'i + offset'in x'i kadar olmalı. Ayrıca random bir offset daha veriyoruz ki tahmin edilebilir olmasın.
          y = player.rect.centery + offset_y + randint(-TILESIZE // 3, TILESIZE // 3) # Her bir flame'in y pozisyonu. Ayrıca random bir offset daha veriyoruz ki tahmin edilebilir olmasın.
          self.animation_player.create_particles('flame',(x,y),groups) # Ve animasyonunu oluşturuyoruz.
          # dict içerisinden 'flame' i alacağız.
          # Konum olarak x ve y'yi verdik.
          # Grup olarak da parametreden gelen değeri veriyoruz.

In [ ]:
# upgrade.py dosyası ile exp ile geliştirme yapacağımız ekranı çizdireceğiz.

# Çalışma mantığı:
# 0-4'e kadar bir index numarası oluşturacağız ve her biri can enerji gibi statları temsil edecek.
# Her bir stat için ayrı bir class oluşturacağız. Her biri için de bir rect.
# Index numarasına göre seçtiğimiz rect'i belli edeceğiz.
# Sol ve sağ tuşları ile bu index numarasını değiştirirken, space tuşu ile de geliştirme işlemini yapacağız.

import pygame
from settings import *

class Upgrade: # Bu işlem için bir class oluşturuyoruz.
  def __init__(self,player): # Ve parametre olarak player'ı veriyoruz.

    # Genel ayarlar
    self.display_surface = pygame.display.get_surface() # Bir display ekranı oluşturduk.
    self.player = player # player'ı da parametre olarak verdik.
    self.attribute_nr = len(player.stats) # Player statları'ının sayısını tutan değişken
    self.attribute_names = list(player.stats.keys()) # Player stats dictionarisi içerisindeki key'leri alıp bir listeye çevirdik.
    self.max_values = list(player.max_stats.values()) # Her bir değerin maksimum değeri. player içerisinden max_stats dict'inin değerlerini list'e çevirip alacağız.
    self.font = pygame.font.Font(UI_FONT, UI_FONT_SIZE) # Ve ekranda göreceğimiz yazı font ve boyutu.

    # Eşya oluşturma
    self.height = self.display_surface.get_size()[1] * 0.8 # Her bir rect boyutu için ekran boyutunun y parametresini aldım ve yüzde 80'ini aldım.
    self.width = self.display_surface.get_size()[0] // 6 # Her bir rect boyutu için ekran boyutunun x parametresini aldım ve 6'ya böldüm. 1 parça da padding olacak.
    self.create_items() # init içerisinde direkt item'ları oluşturan metodu çağırıyoruz.

    # Seçim sistemi
    self.selection_index = 0 # Seçim için gereken index
    self.selection_time = None # Bir timer oluşturmamız lazım çünkü bir kere basıldığında 60 kere tetiklenecek.
    self.can_move = True # Ve tekrar seçim yapabilmemizi sağlayan değişken.

  def input(self): # Kullanıcıdan geliştirme yapmak için input alan metot.
    keys = pygame.key.get_pressed() # Tüm tuşları çağırıyoruz.

    if self.can_move: # Eğer hareket ettirebiliyorsak.
      if keys[pygame.K_RIGHT] and self.selection_index < self.attribute_nr -1 : # Eğer sağ tuşa basıyorsak ve selection_index'imiz toplam attribute sayısının 1 eksiğinden(index hesabı) küçük ise çünkü en sağda isek daha fazla sağ yapamayız. Ayrıca 4'ün üstüne çıkmamızı da engelliyor.
        self.selection_index += 1 # Index'i 1 arttır.
        self.can_move = False # Hareket edebilmeyi False yap ta ki timer onu True yapana kadar.
        self.selection_time = pygame.time.get_ticks() # Timer için süreyi hesaplamaya başlıyoruz.
      elif keys[pygame.K_LEFT] and self.selection_index >= 1: # Eğer sol tuşuna basıyorsak ve selection_index'imiz 1'den büyük ya da eşit ise sola götürebilirsin.
        self.selection_index -= 1 # Index'i 1 azalt.
        self.can_move = False # Hareket edebilmeyi False yap ta ki timer onu True yapana kadar.
        self.selection_time = pygame.time.get_ticks() # Timer için süreyi hesaplamaya başlıyoruz.

      if keys[pygame.K_SPACE]: # Eğer boşluk tuşuna basıyorsak
        self.can_move = False # Hareket edebilmeyi False yap ta ki timer onu True yapana kadar.
        self.selection_time = pygame.time.get_ticks() # Timer için süreyi hesaplamaya başlıyoruz.
        # Timer kullanmazsak tek basışta 60 kere güncelleme yapar.
        self.item_list[self.selection_index].trigger(self.player) # item listemiz içeerisinden indexe göre elemanı seçip trigger metodunu çağırıyoruz.
        # Her space'e basıldığında gerçekleşecek işlem.
    
  def selection_cooldown(self): # Seçim işleminin tekrar gerçekleşebilmesi için gereken timer.
    if not self.can_move: # Eğer hareket ettiremiyor yani bir tuşa basmışsak
      current_time = pygame.time.get_ticks() # Zamanı hesaplamaya başla
      if current_time - self.selection_time >= 300: # Eğer şu anki zamanla hareket tuşuna bastıktan sonra hesaplanan zaman 300 ms'den büyük ise,
        self.can_move = True # Tekrar hareket edebilmeyi True yap.
  
  def create_items(self): # Her bir rect'i oluşturacak metot.
    self.item_list = [] # Tüm rect'leri tutacak olan liste.

    for item,index in enumerate(range(self.attribute_nr)): # Her bir rect için item ve index numarasını aldık. Item nesnesini oluştururken index'i vereceğiz.
      # Yatay pozisyon
      full_width = self.display_surface.get_size()[0] # Tüm ekranın genişliği
      increment = full_width // self.attribute_nr # Artış miktarımız ise toplam genişliğin stat sayısına bölümü
      left = (item * increment) + (increment - self.width) // 2 # Her bir rect arasına ve kenarına offset koyuyoruz.
      # item numarası arttıkça soldan bırakılan boşluk da artıyor. Bu şekilde yan yana oluşturabiliyorum.

      # Dikey pozisyon
      top = self.display_surface.get_size()[1] * 0.1 # Yukarıdan yüzde 10'luk bir boşluk bıraktık çünkü rect boyutu ekranın yüzde 80'i
      
      # Nesnenin oluşturulması
      item = Item(left,top,self.width,self.height,index,self.font) # Parametrelerini vererek bir rect eşyası oluşturuyoruz.
      self.item_list.append(item) # Ve bunu item listesine ekliyoruz.

  def display(self): # Neler yaptığımızı görmek için bir metot oluşturuyoruz.
    self.input() # Input'ları incele
    self.selection_cooldown() # Cooldown'a bak.

    for index, item in enumerate(self.item_list): # Rect item listesindeki her bir item için

      # Değerleri alma
      name = self.attribute_names[index] # Stat ismi
      value = self.player.get_value_by_index(index) # Player içinde oluşturacağım get_value_by_index metodu ile değerleri alacağım.(vereceğim index'e göre)
      max_value = self.max_values[index] # Maksimum değer
      cost = self.player.get_cost_by_index(index) # Player içinde oluşturacağım get_cost_by_index metodu ile değerleri alacağım.(vereceğim index'e göre)

      item.display(self.display_surface,self.selection_index,name,value,max_value,cost) # item nesnesi üzerinden Item class'ı içerisindeki display() metodunu çağırıyoruz ki hepsini çizebilelim.
      # Parametrelerini veriyoruz.

class Item:
  def __init__(self,l,t,w,h,index,font):
    # l soldan alınan boşluk
    # t üstten alınan boşluk
    # w genişliği
    # h yüksekliği
    # index hangi stat'ın rectini seçtiğimiz.
    # font ise font
    self.rect = pygame.Rect(l,t,w,h) # Bir rect oluşturuyoruz ve sol üst genişlik ve uzunluk değerlerine daha önce oluşturduğumuz değerleri veriyoruz.
    self.index = index # Her bir rect'in index numarasını da tutuyoruz.
    self.font = font # Fontumuzu da alıyoruz.

   def display_names(self,surface,name,cost,selected): # Stat isimlerini oluşturacak olan metot.
    # Seçili olma durumu için selected değişkeni oluşturduk.
    
    if selected: # Eğer self.index == selection_num ise
      color = TEXT_COLOR_SELECTED # Text rengini değiştir.
    else: # Değilse
      color = TEXT_COLOR # Aynı kalsın
    
    # Başlık texti
    title_surf = self.font.render(name,False,color) # Başlık alanı için bir surface oluşturduk. Bunu font üzerinden  renderlayarak yaptık. (yazı,AA,renk)
    title_rect = title_surf.get_rect(midtop = self.rect.midtop + pygame.math.Vector2(0,20)) # Ve bir rect oluşturuyoruz.

    # maliyet texti
    cost_surf = self.font.render(f'{int(cost)}',False,color) # Cost alanı için bir surface oluşturduk. Font üzerinden renderladık.
    cost_rect = cost_surf.get_rect(midbottom = self.rect.midbottom - pygame.math.Vector2(0,20)) # Ve rect'ini oluşturuyoruz.

    # çizim
    surface.blit(title_surf,title_rect) # Ve ekranımıza çizdiriyoruz.
    surface.blit(cost_surf,cost_rect) # Ve ekranımıza çizdiriyoruz.

  def display_bar(self,surface,value,max_value,selected):
    # surface hangi yere çizileceğini, value anlık değeri, max_value max değeri, selected ise seçili olma durumunu gösterecek.

    # Çizim ayarları
    top = self.rect.midtop + pygame.math.Vector2(0,60) # Rect'in en yukarısından 60 birim aşağıda.
    bottom = self.rect.midbottom - pygame.math.Vector2(0,60) # Rect'in en aşağısından 60 birim yukarıda.
    color = BAR_COLOR_SELECTED if selected else BAR_COLOR # Eğer seçili veya değilse farklı renkler kullanıyoruz.

    # Bar ayarı
    full_height = bottom[1] - top[1] # bottom ve top'ın y değerlerini alıp çıkardık ki çizginin tüm uzunluğunu bulalım.
    # Ayrıca aşağıdan yukarıyı çıkardım çünkü y ekseni
    relative_number = (value / max_value) * full_height # Eğer 100 canımız varsa ve maks canımız 300 ise ben barın bu çizginin 1/3 'ünde olmasını istiyorum.
    # Ardından bu değeri toplam uzunluk ile çarpıyorum ki pixel'e çevirebilelim.
    value_rect = pygame.Rect(top[0] - 15,bottom[1] - relative_number,30,10) # (left,top,width,height)
    # left için aşağı ya da yukarının x değerini almam yeterli. Bundan da bar yüksekliğinin yarısını çıkarıyorum ki tam ortada olsun.
    # top için ise toplam y değerinden asıl değerimi çıkarıyorum. (aslında en aşağıdan o değer kadar yukarı taşıyorum.)

    # Ekrana çizim
    pygame.draw.line(surface,color,top,bottom,5) # Bir çizgi çiziyoruz. Yukarıdan aşağı ve başlangıç bitiş yerleri bulunan.
    # 5 kalınlığı
    pygame.draw.rect(surface,color,value_rect) # Barımızı çiziyoruz.

  def trigger(self,player): # space'e basıldığında geliştirmeyi yapacak metot.
    upgrade_attribute = list(player.stats.keys())[self.index] # Her space'e basıldığında liste içerisinden index'e göre olan değeri getiriyor.

    if player.exp >= player.upgrade_cost[upgrade_attribute] and player.stats[upgrade_attribute] < player.max_stats[upgrade_attribute]: # Eğer exp'miz yeterli ise ve maks değerin altında isek
      player.exp -= player.upgrade_cost[upgrade_attribute] # Karakter exp'sini cost kadar azalt.
      player.stats[upgrade_attribute] *= 1.2 # Seçili değeri %20 arttırıyoruz.
      player.upgrade_cost[upgrade_attribute] *= 1.4 # Ayrıca istenilen exp değerini de %40 arttırıyoruz.

    if player.stats[upgrade_attribute] > player.max_stats[upgrade_attribute]: # Eğer stat maksimum değerine ulaşırsa
      player.stats[upgrade_attribute] = player.max_stats[upgrade_attribute] # İkisini birbirine eşitle.

  def display(self,surface,selection_num,name,value,max_value,cost): # Ekrana çizen metot.
    # surface üzerine çizeceğimiz yüzeyi tutacak.
    # selection_number seçili rect'in çerçevesini parlatacak
    # name ile hangi stat rect'i olduğunu yazacağız.
    # value ile geçerli olan değeri yazacağız.
    # max_value ile gelişebilecek maksimum değeri yazacağız.
    # cost ile her bir geliştirme bedelini yazacağız.

    if self.index == selection_num: # Eğer indeksimiz seçtiğimiz rect'in indexine eşitse highlighted yap.
      pygame.draw.rect(surface,UPGRADE_BG_COLOR_SELECTED,self.rect) # draw.rect(surface,color,çizilecek rect)
      pygame.draw.rect(surface,UI_BORDER_COLOR,self.rect,4) # draw.rect(surface,color,çizilecek rect,border_width) # Border oluşturduk.
    else: # Eğer değilse normal şekilde dursun
      pygame.draw.rect(surface,UI_BG_COLOR,self.rect) # draw.rect(surface,color,çizilecek rect)
      pygame.draw.rect(surface,UI_BORDER_COLOR,self.rect,4) # draw.rect(surface,color,çizilecek rect,border_width) # Border oluşturduk.

    self.display_names(surface,name,cost,self.index == selection_num) # İkinci parametre True ya da False döndürecek.
    self.display_bar(surface,value,max_value,self.index == selection_num) # Barları gösterecek olan metot